# Converting Homestuck Collection data to an agnostic dataset

This notebook has all the steps for taking data from the [Unofficial Homestuck Collection](https://bambosh.dev/unofficial-homestuck-collection/)'s asset pack and converting it into a dataset that can be used for training a machine learning model.

Some planned uses for this are:
- Summarization
- Style transfer/LORAs
- Chatbots

# Constants and Imports

In [86]:
from IPython.sphinxext.ipython_directive import OUTPUT

"""
IMPORTS
Put all at the beginning because I hate notebooks so much
"""
import base64
import json
import os
import re

import pandas as pd
import requests
from IPython.display import Image
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from openai import OpenAI

ModuleNotFoundError: No module named 'docutils'

In [44]:

"""
CONSTANTS
This will extract constants from the env variables set in the .env file 
and make them accessible to the notebook
"""
print("Loading variables from .env file\n...")
load_dotenv()

ASSET_PACK_FOLDER = os.getenv("ASSET_PACK_FOLDER")
OUTPUT_FOLDER = os.getenv("OUTPUT_FOLDER")
OPENAI_API_KEY = os.getenv("OPENAI_API")
MODEL_ID = os.getenv("MODEL_ID")
print("Loaded variables successfully")

print("Loading constants\n...")
# Relevant folders and files

# Bespoke input files that aren't available elsewhere
INPUT_FOLDER = os.path.join(os.path.dirname(os.path.abspath('')), "input")

# Transcripts and commentary from ReadMSPA, assembled by Bambosh, Makin and Giovanh
MSPA_COMMENTARY = os.path.join(INPUT_FOLDER, "commentary.json")

# Full transcripts from ReadMSPA, assembled by Giovanh and Bambosh
READMSPA_ALTNARRATIVE = os.path.join(INPUT_FOLDER, "altnarrative.json")
READMSPA_ALTTEXT = os.path.join(INPUT_FOLDER, "alttext.json")
READMSPA_TRANSCRIPTS = os.path.join(INPUT_FOLDER, "transcripts.json")

# Panel tags from the Homestuck Search Engine
HSSE_TAGS = os.path.join(INPUT_FOLDER, "hsse_tags.json")
HSSE_SEARCH = os.path.join(INPUT_FOLDER, "hsse_search.json")

# POV cam data folder with txt files
POV_CAM_FOLDER = os.path.join(INPUT_FOLDER, "readable_timelines")

# Crawled data from Acts and Pages of Homestuck
ACTS_AND_PAGES_PART_SUMMARIES = os.path.join(INPUT_FOLDER, 'part_summaries.json')
ACTS_AND_PAGES_ACT_SUMMARIES = os.path.join(INPUT_FOLDER, 'act_summaries.json')
ACTS_AND_PAGES_PAGE_SUMMARIES = os.path.join(INPUT_FOLDER, 'page_summaries.json')

# Homestuck Collection's asset pack data folder
COLLECTION_DATA_FOLDER = os.path.join(ASSET_PACK_FOLDER, "archive/data")

# Holds all the text in MS Paint Adventures, including Homestuck
MSPA_TEXT_JSON = os.path.join(COLLECTION_DATA_FOLDER, "mspa.json")
# Holds the text for news posts
NEWS_JSON = os.path.join(COLLECTION_DATA_FOLDER, "news.json")
# Holds the text for social media posts
SOCIAL_JSON = os.path.join(COLLECTION_DATA_FOLDER, "social.json")
# Most of this is irrelevant, but holds images for additional Hussie comics
# such as Team Special Olympics
ADDITIONAL_COMICS_JSON = os.path.join(COLLECTION_DATA_FOLDER, "comics.json")
# Holds panels
PANELS_FOLDER = os.path.join(ASSET_PACK_FOLDER, "storyfiles")
HS_PANELS_FOLDER = os.path.join(PANELS_FOLDER, "hs2")

print("Loaded constants successfully")

openai_client = OpenAI(api_key=OPENAI_API_KEY)

print(f"OpenAI client loaded with model {MODEL_ID}")

Loading variables from .env file
...
Loaded variables successfully
Loading constants
...
Loaded constants successfully
OpenAI client loaded with model gpt-4o


# Extract MSPA data from Asset Pack

First of all, we want to open the MSPA_TEXT_JSON and fetch all the text data from it. This will be the main source of text data for our dataset.

In [45]:
"""
MSPA_TEXT_JSON format:
{
  "story": {  # The text in MS Paint Adventures, the comics themselves
    "000006": {
      "title": "Look for keyhole",
      "pageId": "000006",
      "timestamp": "1180921880",
      "flag": [],
      "media": [
        "/advimgs/jb/mspaintadventure04.gif",
        "/advimgs/jb/mspaintadventure04b.gif"
      ],
      "content": "",
      "next": [
        "000008"
      ],
      "previous": "000005",
      "theme": "retro"
    },
    "000009": {
      "title": "Loudly tell that guy to pick up key and try it on the door.",
      "pageId": "000009",
      "timestamp": "1180931172",
      "flag": [],
      "media": [
        "/advimgs/jb/mspaintadventure06.gif"
      ],
      "content": "Despite your bellowing, the man casually opens the door and leaves.",
      "next": [
        "000010"
      ],
      "previous": "000008",
      "theme": "retro"
    },
  },...
  "ryanquest": {...},  # Additional Ryanquest comic
  
  "faqs": {
    "general": {
      "title": "General FAQ - MS Paint Adventures",
      "pageId": "general",
      "content": "..." # html
    },
    "new": {...},  # New reader guide
    "science": {...},  # Science FAQ
    "sales": {...}, # This one was probably not Hussie, so ignore
  },
  "psExtras": {...}, # Bonus pages for Problem Sleuth with multiple images, same format with "title", "pageId" and "content" as the subkeys in "faqs"
  "wv": {...}, # "Exile" Homestuck pages with multiple images, same format as psExtras
  # Other keys are fully irrelevant
"""


def get_regular_story_records(mspa_data: dict, section_name: str):
    records = []
    section_data = mspa_data[section_name]
    for page_id, page_content in section_data.items():
        # Extract relevant fields
        record = {
            'section': section_name,
            'pageId': page_id,
            'title': page_content.get('title', ''),
            'content': page_content.get('content', ''),
            'timestamp': page_content.get('timestamp', ''),
            'media': page_content.get('media', []),
            'next': page_content.get('next', []),
            'previous': page_content.get('previous', ''),
            'theme': page_content.get('theme', ''),
            'flag': page_content.get('flag', []),
        }
        records.append(record)
    return records


def get_media_from_html_content(content: str):
    # Extract media from HTML content
    media = []
    # Extract media URLs from the content
    matches = re.findall(r'src="([^"]+)"', content)
    for match in matches:
        # Remove the string "http://www.mspaintadventures.com" and "https://www.mspaintadventures.com" from the beginning if it exists
        match_cleaned = re.sub(r'^https?://www\.mspaintadventures\.com', '', match)
        media.append(match_cleaned)
    return media


def get_onepage_story_records(mspa_data: dict, section_name: str):
    # We need this one for faqs/psExtras/wv, which keep all information in one page and need the panels manually parsed
    records = []
    section_data = mspa_data[section_name]
    for page_id, page_content in section_data.items():
        # Extract relevant fields
        record = {
            'section': section_name,
            'pageId': page_id,
            'title': page_content.get('title', ''),
            'content': page_content.get('content', ''),
            'timestamp': None,
            'media': get_media_from_html_content(page_content.get('content', '')),
            'next': None,
            'previous': None,
            'theme': None,
            'flag': None,
        }
        records.append(record)
    return records


# Function to load and process MSPA text data from mspa.json
def load_mspa_text_data(mspa_json_path):
    """
    Loads and processes MSPA text data from the given JSON file.

    Args:
        mspa_json_path (str): Path to the mspa.json file.

    Returns:
        pd.DataFrame: A DataFrame containing the MSPA text data.
    """
    import json
    import pandas as pd

    # Load the JSON data
    with open(mspa_json_path, 'r', encoding='utf-8') as f:
        mspa_data = json.load(f)

    # Initialize an empty list to store records
    records = []

    # Print unique sections
    print(f"Unique sections in mspa.json: {list(mspa_data.keys())}")

    # Sections to process (basically we just skip FAQs)
    records += get_regular_story_records(mspa_data, 'story')
    records += get_regular_story_records(mspa_data, 'ryanquest')
    records += get_onepage_story_records(mspa_data, 'wv')
    records += get_onepage_story_records(mspa_data, 'psExtras')

    # Create a DataFrame from the records
    df = pd.DataFrame(records)

    return df


# Now call the function and store the result in a DataFrame
mspa_df = load_mspa_text_data(MSPA_TEXT_JSON)
mspa_df

Unique sections in mspa.json: ['story', 'ryanquest', 'psExtras', 'wv', 'faqs', 'damara', 'oilRetcon']


,section,pageId,title,content,timestamp,media,next,previous,theme,flag
0,story,000006,Look for keyhole,,1180921880,"[/advimgs/jb/mspaintadventure04.gif, /advimgs/...",[000008],000005,retro,[]
1,story,000009,Loudly tell that guy to pick up key and try it...,"Despite your bellowing, the man casually opens...",1180931172,[/advimgs/jb/mspaintadventure06.gif],[000010],000008,retro,[]
2,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,"[/advimgs/jb/mspaintadventure07.gif, /advimgs/...",[000011],000009,retro,[]
3,story,000011,Open door,Wonders where the fuck that pumpkin went???,1180931414,[/advimgs/jb/mspaintadventure08.gif],[000012],000010,retro,[]
4,story,000005,Grow two muscular arms with which the manhandl...,"He already has arms, stupid!",1180921870,[/advimgs/jb/mspaintadventure03.gif],[000006],000004,retro,[]
...,...,...,...,...,...,...,...,...,...,...
10049,psExtras,ps000037,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000037_1.gif, /extras/ps000037_2.gi...",None,None,None,None
10050,psExtras,ps000039,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000039_1.gif, /extras/ps000039_2.gif]",None,None,None,None
10051,psExtras,ps000034,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000034_1.gif, /extras/ps000034_2.gi...",None,None,None,None
10052,psExtras,ps000031,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000031_1.gif, /extras/ps000031_2.gi...",None,None,None,None


In [46]:
# Show pageId == anunsealedrecord, a special case
mspa_df[mspa_df['pageId'] == 'anunsealedtunnel']

,section,pageId,title,content,timestamp,media,next,previous,theme,flag
10006,wv,anunsealedtunnel,MS Paint Adventures,"\n\n <tbody><tr>\n <td bgcolor=""#FFFFFF""...",None,[/storyfiles/hs2/waywardvagabond/anunsealedtun...,None,None,None,None


# Creating vizPageId from pageId

The Homestuck Collection uses the MSPA page IDs, but the Homestuck website uses its own page IDs. We can convert the MSPA page IDs to the Homestuck website page IDs with some clever and totally not bruteforced heuristics. We will need to do this sooner or later because multiple upcoming datasets use the Viz page ids, which are missing in mspa_df right now.

In [47]:
# Define the adventure number to story name mapping
adventure_mapping = {
    0: "Other Section",
    1: "Jailbreak",
    2: "Bard Quest",
    3: "Blood Spade",
    4: "Problem Sleuth",
    5: "Homestuck BETA",
    6: "Homestuck",
    "ryanquest": "Ryanquest",
    "psExtras": "Problem Sleuth Extras"
}

# Define the story name to URL path mapping
story_url_mapping = {
    "Jailbreak": "jailbreak",
    "Bard Quest": "bard-quest",
    "Blood Spade": "blood-spade",
    "Problem Sleuth": "problem-sleuth",
    "Homestuck BETA": "beta",
    "Homestuck": "story",
    "Ryanquest": "ryanquest",
    "Other Section": "story/waywardvagabond"
}


def get_story_num(page_id):
    """
    Determines the story ID based on the page number.
    
    Args:
        page_id (str): The pageId, can be numeric or special identifier.
    
    Returns:
        int or str: The story number (1-6) or 'Unknown'
    """
    if page_id.isdigit():
        page_num = int(page_id.lstrip('0'))
        if page_num <= 135:
            return 1  # Jailbreak
        elif 136 <= page_num <= 216:
            return 2  # Bard Quest
        elif 219 <= page_num <= 1892:
            return 4  # Problem Sleuth
        elif 1893 <= page_num <= 1900:
            return 5  # Homestuck BETA
        elif 1901 <= page_num <= 10030:
            return 6  # Homestuck
        else:
            return "Unknown"
    else:
        # Handle special cases
        if page_id == 'mc0001':
            return 3  # Blood Spade
        elif page_id in ['pony', 'pony2', 'darkcage', 'darkcage2']:
            return 6  # Homestuck
        else:
            return "Unknown"


def mspa_to_viz(row):
    """
    Converts a row's pageId and section to vizStory, vizPageNumber, vizPageId.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        pd.Series: A series with 'vizStory', 'vizPageNumber', and 'vizPageId'.
    """
    section = row['section'].lower()
    page_id = row['pageId']

    if section == 'ryanquest':
        viz_story = "Ryanquest"
        try:
            viz_page_number = int(page_id.lstrip('0'))
        except:
            viz_page_number = page_id  # Preserve special strings if any
        viz_page_id = f"ryanquest/{viz_page_number}" \
            if isinstance(viz_page_number, int) else f"ryanquest/{viz_page_number}"
    elif section == 'story':
        story_num = get_story_num(page_id)
        if isinstance(story_num, int) and story_num in adventure_mapping:
            viz_story = adventure_mapping[story_num]
            # Adjust vizPageNumber based on story_num
            try:
                if story_num == 1:  # Jailbreak
                    viz_page_number = int(page_id.lstrip('0'))
                elif story_num == 2:  # Bard Quest
                    viz_page_number = int(page_id.lstrip('0')) - 135
                elif story_num == 3:  # Blood Spade
                    viz_page_number = "1"  # Only 'mc0001'
                elif story_num == 4:  # Problem Sleuth
                    viz_page_number = int(page_id.lstrip('0')) - 218
                elif story_num == 5:  # Homestuck BETA
                    viz_page_number = int(page_id.lstrip('0')) - 1892
                elif story_num == 6:  # Homestuck
                    if page_id in ['pony', 'pony2', 'darkcage', 'darkcage2']:
                        viz_page_number = page_id  # Preserve special strings
                    else:
                        viz_page_number = int(page_id.lstrip('0')) - 1900
                else:
                    viz_page_number = None
            except:
                viz_page_number = None

            # Construct vizPageId based on vizPageNumber
            if isinstance(viz_page_number, int):
                viz_page_id = f"{story_num}/{viz_page_number}"
            elif isinstance(viz_page_number, str):
                viz_page_id = f"{story_num}/{viz_page_number}"
            else:
                viz_page_id = f"{story_num}/000000"
        else:
            viz_story = "Unknown Adventure"
            viz_page_number = None
            viz_page_id = "Unknown/000000"
    else:
        viz_story = "Other Section"
        try:
            viz_page_number = int(page_id.lstrip('0'))
        except:
            viz_page_number = page_id  # Preserve special strings if any
        viz_page_id = f"other/{viz_page_number}" if isinstance(viz_page_number, int) else f"other/{viz_page_number}"

    return pd.Series({
        'vizStory': viz_story,
        'vizPageNumber': viz_page_number,
        'vizPageId': viz_page_id
    })


def create_viz_link(row):
    """
    Creates a visualization link based on vizStory and vizPageNumber.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        str or None: The constructed URL or None if information is missing.
    """
    viz_story = row['vizStory']
    viz_page_number = row['vizPageNumber']

    if pd.isnull(viz_story) or pd.isnull(viz_page_number):
        return None

    base_url = "https://homestuck.com/"
    story_path = story_url_mapping.get(viz_story, "other")

    # Handle potential edge cases where viz_page_number is not an integer
    if isinstance(viz_page_number, float):
        if viz_page_number.is_integer():
            viz_page_number = int(viz_page_number)
        else:
            # If it's not an integer, handle accordingly (e.g., set to None or round)
            return None

    return f"{base_url}{story_path}/{viz_page_number}"


mspa_df['pageId'] = mspa_df['pageId'].astype(str)

# Apply the mspa_to_viz function to create 'vizStory', 'vizPageNumber', and 'vizPageId' columns
mspa_df[['vizStory', 'vizPageNumber', 'vizPageId']] = mspa_df.apply(mspa_to_viz, axis=1)

# Create the 'vizLink' column
mspa_df['vizLink'] = mspa_df.apply(create_viz_link, axis=1)


# Convert 'vizPageNumber' to integer where possible, else keep as string for special cases
def convert_viz_page_number(x):
    if isinstance(x, int):
        return x
    elif isinstance(x, str):
        return x  # Preserve special strings
    else:
        return pd.NA


mspa_df['vizPageNumber'] = mspa_df['vizPageNumber'].apply(convert_viz_page_number)

# Display the updated DataFrame sample
mspa_df[['section', 'title', 'pageId', 'content', 'vizStory', 'vizPageNumber', 'vizPageId', 'vizLink']].sample(10)

,section,title,pageId,content,vizStory,vizPageNumber,vizPageId,vizLink
9569,story,[A6A6I5] ====>,009608,,Homestuck,7708,6/7708,https://homestuck.com/story/7708
6803,story,==>,006845,"After a lot of bullshitting around, you calmly...",Homestuck,4945,6/4945,https://homestuck.com/story/4945
2645,story,==>,002684,And finally a ripe yellow EUREKA LEMON.,Homestuck,784,6/784,https://homestuck.com/story/784
7471,story,==>,007512,,Homestuck,5612,6/5612,https://homestuck.com/story/5612
6381,story,Dirk: Examine wardrobe.,006424,You tell Squarewave to scoot out of the way so...,Homestuck,4524,6/4524,https://homestuck.com/story/4524
5421,story,Be future Jack Noir.,005463,You are now future Jack Noir.<br /><br />Prese...,Homestuck,3563,6/3563,https://homestuck.com/story/3563
4610,story,==>,004650,"Unsurprisingly, you return to a room full of c...",Homestuck,2750,6/2750,https://homestuck.com/story/2750
4381,story,It is time.,004425,It's the voice again.<br /><br />You were wond...,Homestuck,2525,6/2525,https://homestuck.com/story/2525
8240,story,[A6A6I1] ====>,008287,"|DIALOGLOG|<br /><span style=""color: #4ac925"">...",Homestuck,6387,6/6387,https://homestuck.com/story/6387
1002,story,PS+AD: Go up.,001041,The path is blocked! You'll need an awful stro...,Problem Sleuth,823,4/823,https://homestuck.com/problem-sleuth/823


# Unpacking the page-based dataset into a panel-based dataset

The previous works fine for data sourced on pages instead of panels, like commentaries, but we're merging with ReadMSPA next, so we will need to `explode` the data and do some more edits, since the panel srcs will need some editing to match the transcripts.

In [48]:
mspa_panels_df = mspa_df.explode('media').reset_index(drop=True)
# Rename 'media' to 'src' for clarity
mspa_panels_df.rename(columns={'media': 'src'}, inplace=True)
# Get a sample of Homestuck panels
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6


In [49]:
# Some panel transcripts will have an id format like `00001.gif.transcript`, which in our current src is `/storyfiles/hs2/00001.gif`. We need to make a transcriptSrc to match the panel src. This applies no matter what the comics is, so we can't assume it will be /storyfiles/hs2/... we just pick the actual image file name and add '.transcript' to it.
def create_transcript_src(src):
    if pd.isnull(src):
        return None

    # Extract the image file name from the src
    image_file = src.split('/')[-1]
    # Append '.transcript' to the image file name
    transcript_src = f"{image_file}.transcript"

    return transcript_src


# Apply the function to create 'transcriptSrc' column
mspa_panels_df['transcriptSrc'] = mspa_panels_df['src'].apply(create_transcript_src)
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,transcriptSrc
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2,00002.gif.transcript
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1,00001.gif.transcript
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4,00004.gif.transcript
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3,00003.gif.transcript
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8,00008.gif.transcript
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5,00005.gif.transcript
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9,00009.gif.transcript
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_1.gif.transcript
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_2.gif.transcript
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6,00006.gif.transcript


In [50]:
# Now we need to do something similar for MSPA URLs to match the key format for pages of the READMSPA format. We have a pageId like 001901, but we need to get the story_num using the matching we already have, and prepend that to the page number. So 001901 would become '6/001901' because that page number matches Homestuck. This can be the new column "pageNumber" because it's the vizPageNumber in MSPA format..
def create_page_number(row):
    """
    Creates a page number based on vizStory and vizPageNumber.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        str or None: The constructed page number or None if information is missing.
    """

    viz_page_number = row['pageId']

    if pd.isnull(viz_page_number):
        return None

    viz_story_number = get_story_num(str(viz_page_number))

    return f"{viz_story_number}/{viz_page_number}"


# Apply the function to create 'pageNumber' column
mspa_panels_df['pageNumber'] = mspa_panels_df.apply(create_page_number, axis=1)

# Display the updated DataFrame sample
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,transcriptSrc,pageNumber
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2,00002.gif.transcript,6/001902
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1,00001.gif.transcript,6/001901
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4,00004.gif.transcript,6/001904
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3,00003.gif.transcript,6/001903
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8,00008.gif.transcript,6/001908
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5,00005.gif.transcript,6/001905
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9,00009.gif.transcript,6/001909
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_1.gif.transcript,6/001907
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_2.gif.transcript,6/001907
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6,00006.gif.transcript,6/001906


# Getting panel transcripts from ReadMSPA plugin

ReadMSPA's data (and its plugin from the collection) comes with text transcripts of every image's text, if not descriptions. We can use that.

In [51]:
"""
The ReadMSPA data is divided in three similar files for unknown reasons, READMSPA_TRANSCRIPTS, READMSPA_ALTTEXT, READMSPA_ALTNARRATIVE. The latter tends to have way longer transcriptions, but there are long transcriptions in READMSPA_ALTTEXT, and they have different formats besides. READMSPA_TRANSCRIPTS even contains transcripts of youtube links linked in the pages, which we should preserve anyway. 
Note the keys are not equivalent to pageId. These keys use the mspaintadventures.com format, where page numbers were padded with zeroes, as well as sharing a count for every adventure in the site, from Jailbreak to Homestuck. Homestuck's first page is 001901 and it's the adventure number 6 (instead of 4 for historical reasons, but this is not really relevant to us right now).

Portion of READMSPA_TRANSCRIPTS (the key is a pageNumber, and the array is further discriminated by the id property, which will match transcriptSrc):
```json
{
    "6/001901": [
        {
            "src": null,
            "alt": "SBURB BETA",
            "contents": "SBURB BETA",
            "id": "00001.gif.transcript",
            "data": null,
            "class": [
                "transcript"
            ],
            "style": "background: white; color: #C1C1C1",
            "tag": "div"
        }
    ],
    "6/010030": [
        {
            "src": "http/www.youtube.com-rmzu89jy2j8.mp4",
            "alt": "* (This transcribes both the snapchat pics from the right of the video - which as narrative, are included in readmspa.org word count stats - and the credits from the left, which are not.)\n\n\nmspaofficial\n\nAdded Me\nAdd Friends\nMy Friends\n\n\n* HOMESTUCK\n\n* BY\n* ANDREW HUSSIE\n\n\nGreetings From\nEARTH C\n\n\n* SOUND\n* CONTRIBUTORS (A-Z)\n\n* Alex Rosetti\n* Andrew Huo\n* BurnedKirby\n* Charlie Clouser\n* Clark Powell\n* Curt Blakeslee\n* David DeCou\n* David Ko\n\n\nMAYOR\n\n\n* Dianne Warren\n* DJ Sai Tae\n* Eddie Morton\n* Eligecos\n* Erik Scheele\n* Eston Schweickart\n* First Turn Fold\n* Frank Haught\n* Gabe Nezovic\n\n\n* Insane Clown Posse\n* James Roach\n* Jan Van Den Hemel\n* Joseph Aylsworth\n* Kalibration\n* Kera Jones\n* Kevin Regamey\n* Kezinox\n* Malcolm Brown\n\n\n* Mark Hadley\n* Michael Guy Bowman\n* Michael Vallejo\n* Nick Smalley\n* Noel Sadwin\n* Paul Tuttle Starr\n* Perry Sullivan\n* Plumegeist\n* Robert J! Lake\n\n\n* Samm 413\ntentacleTherapist\ngallowsCallibrator\ntipsyGnostalgic\ngrimAuxiliatrix\ngolgothasTerror\ntimaeusTestified\ngutsyGumshoe\n\n\nCALIBORN: COME AT ME BRO.\n\n\nJOHN: step off.\n\n\nCALIBORN: COME TO MY DARK CARNIVAL. \"MOTHER FUCKER\".\n\n\nJOHN: i'll do it...\n\n\nCALIBORN: MAKE A MOVE. AND THE BUNNY GETS IT.",
            "contents": "",
            "id": "08123.mp4",
            "data": "End Credits",
            "class": [
                "flash",
                "alt-narrative"
            ],
            "style": null,
            "tag": "embed"
        }
    ],
```

Portion of READMSPA_ALTTEXT (a key matches the src column):
```json
{
    "advimgs/jb/mspaintadventure06.gif": "\nBELLOW!\n\nSLAM\n",
    "advimgs/jb/mspaintadventure09d.gif": "CRACK",
    "advimgs/jb/mspaintadventure09e.gif": "SLAM",
    "advimgs/jb/mspaintadventure05.gif": "CLANK",
    "advimgs/jb/mspaintadventure08.gif": "? ? ? ?",
    ...
    "storyfiles/hs2/07655.gif": "\nDAVE: ...\nDIRK: ...\n",
    "storyfiles/hs2/07651.gif": "nuzzzzzzzzzle",
    "storyfiles/hs2/07650.gif": "nuzzzzle",
    "storyfiles/hs2/07648.gif": "KANAYA: Shoutpole",
}

Portion of READMSPA_ALTNARRATIVE (the key is a pageNumber, and the array is further discriminated by the id property, which will match transcriptSrc):
```json
"6/002153": [
        {
            "src": null,
            "alt": "<style>.page[id=\"6/002153\"] .walkaround p { border-color: #23CE27; border-radius: 0;\n\t\t       \t    \t\t    background-color: white; max-width: 100%; }</style>\n* Derived in part from <a href=\"http://pastebin.com/a3k6RgMR\">a transcript by ShadowOfFate</a>\n\n<div style=\"padding: 1em; background-color: white; border: solid 3px black\">CLICK THIS</div><dl>\n<dt>* click it!</dt>\n<dd>\n<p>To walk around, use the mouse, arrow keys, or WASD keys. Click on various objects to open command menus for them!\n\nOutstanding Flash programming by Alexis 'Gankro' Beingessner.</p>\n</dd>\n\n<dt>* click jestersprite</dt>\n<dd>\n<kbd>&gt; WHAT'S THAT\n</kbd>\n<p>It looks different now.\n\nAfter you bit that APPLE, your whole house seemed to be trasported somewhere. Then the APPLE disappeared and the KERNELSPRITE underwent a transformation. Aside from the change in appearance, the transformation doesn't seem to have any relevant ramifications. You still can't understand a word this idiot says.</p>\n<kbd>&gt; THE GHOST CLOWN...",
            "id": "00253.swf.transcript",
            "data": "John explores his house with WV (transcript and walkthrough)"
        }
    ],
    "6/002149": [
        {
            "src": null,
            "alt": "* Derived from <a href=\"http://mspaintadventures.wikia.com/wiki/Rose%27s_Walkthrough\">MSPA Wiki</a>\n\n<p style=\"background-color: black; color: white; padding: 1em;\n          font-family: Lucida Console, courier;\"><b>Sburb Beta FAQ/Walkthrough</b> by <span style=\"color: #ff6600\">tentacleTherapist</span>\n<b>Hosted by</b> <span style=\"color: #ff6600\">GameFAQs</span>\nVersion 1.0, Last Updated 2009-04-13\n<span style=\"color: #ff6600\">View/Download Original File</span>\nLiked this FAQ? Click to <span style=\"color: #ff6600\">recommend</span> it to other GameFAQs users.\n</p>\n<article style=\"font-family: Lucids Console, courier\">\nSburb Beta Walkthrough\nVersion 1.0, April 13, 2009\nBy tentacleTherapist\n\n=============================== TABLE OF CONTENTS ===============================\n\n1. Caveats and Condolences........................... [0000]\n2. Walkthrough (Incomplete).......................... [A000]\n2.1. An Examination of the Basics.................... [A000]\n2.2. So your cruxtruder is ticking. Do this to live.. [A100]\n2.3. The Long and Short. The Medium too.............. [B100]\nsome stuff about captcha codes and punch card alchemy [Z001]\nC. Appendix 3 -- Screen Captures, pt. 1.............. [Z301]\n?. Rose: Egress...",
            "id": "00249_2.gif.transcript",
            "data": "Rose's Sburb Beta Walkthrough - Caveats and Condolences"
        }
    ],
    "6/002037": [
        {
            "src": null,
            "alt": "* Derived from <a href=\"http://pastebin.com/FQhpyeKx\">a transcript by ShadowOfFate</a>\n\n<ul style=\"list-style: none; background-color: #4CE24E; padding: 1em;\n    color: white; text-align: center; white-space: normal\">\n<li>Transforming Soffits</li>\n<li>Reorganizing Keys</li>\n<li>Formalizing Immersion Joints</li>\n<li>Justifying Kick Extractors</li>\n<li>Advising Aggregates</li>\n<li>Managing Elbows</li>...",
            "id": "00137.swf.transcript",
            "data": "Sburb Beta loading screen"
        }
    ]
}

Our objective here is to combine the information so that, for each panel, we'll have its text transcript.
"""


# Extract the ReadMSPA data from the JSON files
def load_readmspa_data(transcripts_path, alttext_path, altnarrative_path):
    """
    Loads and processes the ReadMSPA data from the given JSON files.

    Args:
        transcripts_path (str): Path to the transcripts JSON file.
        alttext_path (str): Path to the alttext JSON file.
        altnarrative_path (str): Path to the altnarrative JSON file.

    Returns:
        pd.DataFrame: A DataFrame containing the ReadMSPA data.
    """
    # Load the JSON data
    with open(transcripts_path, 'r', encoding='utf-8') as f:
        transcripts_data = json.load(f)
    with open(alttext_path, 'r', encoding='utf-8') as f:
        alttext_data = json.load(f)
    with open(altnarrative_path, 'r', encoding='utf-8') as f:
        altnarrative_data = json.load(f)

    # Initialize an empty list to store page records
    page_records = []

    # Iterate over each data dictionary of READMSPA_TRANSCRIPTS and READMSPA_ALTNARRATIVE
    for data_dict in [transcripts_data, altnarrative_data]:
        # Iterate over each key-value pair
        for key, value_list in data_dict.items():
            # Extract relevant fields
            for value in value_list:
                record = {
                    'pageNumber': key,
                    'src': value.get('src', None),
                    'contents': value.get('contents', ''),
                    'transcriptSrc': value.get('id', ''),
                    'data': value.get('data', ''),
                    'class': value.get('class', []),
                    'style': value.get('style', ''),
                    'tag': value.get('tag', ''),
                    'alt': value.get('alt', ''),
                }
                page_records.append(record)

    # Initialize an empty list to store alttext records
    panel_records = []

    # Iterate over each key-value pair in READMSPA_ALTTEXT
    for key, value in alttext_data.items():
        # Extract relevant fields
        record = {
            'src': key,
            'contents': value,
        }
        panel_records.append(record)

    # Create DataFrames from the records
    page_df = pd.DataFrame(page_records)
    panel_df = pd.DataFrame(panel_records)

    return page_df, panel_df


# Now call the function and store the results in DataFrames
readmspa_pages_df, readmspa_panels_df = load_readmspa_data(READMSPA_TRANSCRIPTS, READMSPA_ALTTEXT,
                                                           READMSPA_ALTNARRATIVE)

# Display the updated DataFrame samples
readmspa_pages_df.sample(5)


,pageNumber,src,contents,transcriptSrc,data,class,style,tag,alt
99,6/006727,None,"<style>.page[id=""6/006727""] .transcript p { ba...",04824.swf.transcript,"MystStuck, Part 3 - Gamzee's potions (transcri...","[transcript, alt-narrative]",,div,"<style>.page[id=""6/006727""] .transcript p { ba..."
174,6/007626,storyfiles/hs2/05723/05723.swf,,05723.swf,Help Jane Investigate!,[],,,HELP JANE INVESTIGATE!\n\nDANCING ANIMATION TO...
64,6/008392,None,"<div class=""log""><section>\n<span style=""color...",06489_2_retcon.gif.transcript,horsesuck,[transcript],,div,"<div class=""log""><section>\n<span style=""color..."
220,6/002172,None,,00272_2.gif.transcript,Rose's Sburb Beta Walkthrough - An Examination...,[],,,"* Derived from <a href=""http://mspaintadventur..."
4,6/009763,None,"<p style=""text-transform: uppercase; text-alig...",07859_1.gif.transcript,None,[transcript],,div,"<p style=""text-transform: uppercase; text-alig..."


In [52]:
readmspa_panels_df.sample(5)

,src,contents
869,storyfiles/hs2/04818.gif,Alert : ⚕
2494,storyfiles/hs2/00728.gif,1 → 2 → 3
233,storyfiles/hs2/06811.gif,clown suplex
1832,storyfiles/hs2/02296.gif,\nAlert : ♋ \nAlert : ♏ \n
3245,advimgs/ps/ps236.gif,PUSH


In [53]:
# Now we can merge readmspa_pages_df on its id (which will match transcriptSrc) with mspa_panels_df on transcriptSrc
# Note we only care about 
mspa_panels_df = mspa_panels_df.merge(readmspa_pages_df, on='transcriptSrc', how='left',
                                      suffixes=('', '_readmspapages'))
# Drop duplicate rows, we just want one record per "src".
mspa_panels_df.drop_duplicates(subset='src', inplace=True)
# Show some panels with transcripts
mspa_panels_df[~mspa_panels_df['contents'].isnull()].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,transcriptSrc,pageNumber,pageNumber_readmspapages,src_readmspapages,contents,data,class,style,tag,alt
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],...,00002.gif.transcript,6/001902,6/001902,None,"<b style=""color: black"">ZOOSMELL POOPLORD</b>\...",None,[transcript],font-family: courier; background: white,div,"<b style=""color: black"">ZOOSMELL POOPLORD</b>\..."
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],...,00001.gif.transcript,6/001901,6/001901,None,SBURB BETA,None,[transcript],background: white; color: #C1C1C1,div,SBURB BETA
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],...,00003.gif.transcript,6/001903,6/001903,None,"<b style=""color: green"">✓</b> <b style=""color:...",None,[transcript],font-family: courier; background: white,div,"<b style=""color: green"">✓</b> <b style=""color:..."
2408,story,001912,John: Read note on drawer.,This note is rich with the aromas of FATHERLY ...,1239679025,/storyfiles/hs2/00012.gif,[001913],001911,,[],...,00012.gif.transcript,6/001912,6/001912,None,HAPPY BIRTHDAY SON.\n\nI AM SO PROUD OF YOU.,None,"[transcript, dad-note]",,div,HAPPY BIRTHDAY SON.\n\nI AM SO PROUD OF YOU.
2562,story,002037,[S] ==>,,1241865178,/storyfiles/hs2/00137/00137.swf,[002038],002036,,"[F, HQ]",...,00137.swf.transcript,6/002037,6/002037,None,"* Derived from <a href=""http://pastebin.com/FQ...",Sburb Beta loading screen,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://pastebin.com/FQ..."
2694,story,002149,ACT 2 ==>,,1244666782,/storyfiles/hs2/00249_2.gif,[002150],002148,,[],...,00249_2.gif.transcript,6/002149,6/002149,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - Caveats and Co...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2704,story,002153,[S] YOU THERE. BOY.,,1244949337,/storyfiles/hs2/00253/00253.swf,[002154],002152,,[F],...,00253.swf.transcript,6/002153,6/002153,None,"<style>.page[id=""6/002153""] .walkaround p { bo...",John explores his house with WV (transcript an...,"[transcript, alt-narrative, walkaround]",,div,"<style>.page[id=""6/002153""] .walkaround p { bo..."
2726,story,002172,==>,,1245374550,/storyfiles/hs2/00272_2.gif,[002173],002171,,[],...,00272_2.gif.transcript,6/002172,6/002172,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - An Examination...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2919,story,002340,==>,"She's not finished with this yet! Jeez, cut he...",1249444721,/storyfiles/hs2/00440_2.gif,[002341],002339,,[],...,00440_2.gif.transcript,6/002340,6/002340,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - The Long and S...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2998,story,002410,==>,"<a href=""http://www.mspaintadventures.com/stor...",1250822032,/storyfiles/hs2/00510_2.gif,[002411],002409,,[],...,00510_2.gif.transcript,6/002410,6/002410,None,"<article style=""font-family: Lucida Console, c...",Rose's Sburb Beta Walkthrough - Screen Captures,"[transcript, alt-narrative]",,div,"<article style=""font-family: Lucida Console, c..."


In [54]:
# Merge the panel data too, this time on readmspa_panels_df's src, which matches mspa_panels_df's src, but, critically, without the slash at the beginning. 
# Make a copy of the src column in readmspa_panels_df to match the src column in mspa_panels_df
mspa_panels_df['src_copy'] = mspa_panels_df['src'].apply(lambda x: x[1:] if x.startswith('/') else x)
# Merge the DataFrames
mspa_panels_annotated_df = mspa_panels_df.merge(readmspa_panels_df, left_on='src_copy', right_on='src', how='left',
                                                suffixes=('', '_readmspanels'))
# Show a panel with annotations
mspa_panels_annotated_df[mspa_panels_annotated_df['src'] == '/storyfiles/hs2/01085_2.gif']

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,src_readmspapages,contents,data,class,style,tag,alt,src_copy,src_readmspanels,contents_readmspanels
3653,story,002985,Dave: Eject your modus and set it to Scrabble ...,You dump all this crap all over the roof.<br /...,1262302360,/storyfiles/hs2/01085_2.gif,[002986],002984,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,storyfiles/hs2/01085_2.gif,storyfiles/hs2/01085_2.gif,\n[EJECT]\n\nhash functions\nA=1; B=3; C=3; D=...


In [55]:
# Time to clean up, now that we know transcripts in either "alt" or "contents_readmspanels"
# Useless columns are "transcriptSrc", "pageNumber_readmspapages", "src_readmspapages", "contents", "data", "src_copy", "src_readmspanels"
# "class", "style" and "tag" should be renamed to "transcript_class", "transcript_style" and "transcript_tag" respectively
# "transcript" should be a new column that contains the value of either "alt" or "contents_readmspanels"
# "alt" and "contents_readmspanels" should be dropped afterwards

# Drop the unnecessary columns
mspa_panels_annotated_df.drop(
    columns=['transcriptSrc', 'pageNumber_readmspapages', 'src_readmspapages', 'contents', 'data', 'src_copy',
             'src_readmspanels'], inplace=True)

# Rename the 'class', 'style', and 'tag' columns
mspa_panels_annotated_df.rename(
    columns={'class': 'transcript_class', 'style': 'transcript_style', 'tag': 'transcript_tag'}, inplace=True)

# Create the 'transcript' column
mspa_panels_annotated_df['transcript'] = mspa_panels_annotated_df['alt'].fillna(
    mspa_panels_annotated_df['contents_readmspanels'])

# Drop the 'alt' and 'contents_readmspanels' columns
mspa_panels_annotated_df.drop(columns=['alt', 'contents_readmspanels'], inplace=True)

# Show a panel with cleaned annotations
mspa_panels_annotated_df[mspa_panels_annotated_df['src'] == '/storyfiles/hs2/01085_2.gif']

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript
3653,story,002985,Dave: Eject your modus and set it to Scrabble ...,You dump all this crap all over the roof.<br /...,1262302360,/storyfiles/hs2/01085_2.gif,[002986],002984,,[],Homestuck,1085,6/1085,https://homestuck.com/story/1085,6/002985,NaN,NaN,NaN,\n[EJECT]\n\nhash functions\nA=1; B=3; C=3; D=...


In [56]:
mspa_panels_annotated_df

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript
0,story,000006,Look for keyhole,,1180921880,/advimgs/jb/mspaintadventure04.gif,[000008],000005,retro,[],Jailbreak,6,1/6,https://homestuck.com/jailbreak/6,1/000006,NaN,NaN,NaN,NaN
1,story,000006,Look for keyhole,,1180921880,/advimgs/jb/mspaintadventure04b.gif,[000008],000005,retro,[],Jailbreak,6,1/6,https://homestuck.com/jailbreak/6,1/000006,NaN,NaN,NaN,NaN
2,story,000009,Loudly tell that guy to pick up key and try it...,"Despite your bellowing, the man casually opens...",1180931172,/advimgs/jb/mspaintadventure06.gif,[000010],000008,retro,[],Jailbreak,9,1/9,https://homestuck.com/jailbreak/9,1/000009,NaN,NaN,NaN,\nBELLOW!\n\nSLAM\n
3,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,/advimgs/jb/mspaintadventure07.gif,[000011],000009,retro,[],Jailbreak,10,1/10,https://homestuck.com/jailbreak/10,1/000010,NaN,NaN,NaN,NaN
4,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,/advimgs/jb/mspaintadventure07b.gif,[000011],000009,retro,[],Jailbreak,10,1/10,https://homestuck.com/jailbreak/10,1/000010,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11691,psExtras,ps000031,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000031_3.gif,None,None,None,None,Other Section,ps000031,other/ps000031,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000031,NaN,NaN,NaN,NaN
11692,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_1.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN
11693,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_2.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN
11694,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_3.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN


# Getting panel tags from HSSE

The Homestuck Search Engine people tagged the tags of the first four acts, from characters to locations and more. Will be extremely useful for image transcription.

In [57]:
"""
HSSE_TAGS and HSSE_SEARCH file contain the entirety of the Homestuck Search Engine tagged data (only the first four acts, until page 1988 inclusive and excluding some swfs) in its own bespoke JSON format.
HSSE_TAGS is the simpler json with tag definitions, and which tags contain other tags:
```json
 "definitions": {
    "0": {
      "_id": 0,
      "name": "Character",
      "children": [
        1,
        32,
        56,
        60,
        104,
        132,
        148,
        155,
        173,
        184,
        253
      ]
    },
    "1": {
      "_id": 1,
      "name": "Human",
      "children": [
        2,
        15
      ]
    },
    "2": {
      "_id": 2,
      "name": "Kid",
      "children": [
        3,
        10
      ]
    },
    "3": {
      "_id": 3,
      "name": "Beta Kid",
      "children": [
        4,
        5,
        7,
        9
      ]
    },
    "4": {
      "_id": 4,
      "name": "John Egbert",
      "children": []
    },
    ...
}
```

HSSE_SEARCH is the more complex json with the actual tags for each panel:
```json
[
  {
    "_id": 0,
    "type": 0,
    "content": "https://www.homestuck.com/images/storyfiles/hs2/00001.gif",
    "thumbnail": "https://www.homestuck.com/images/storyfiles/hs2/00001.gif",
    "url": "https://homestuck.com/story/1",
    "tags": [
      1384,
      1385,
      391,
      321,
      4,
      749,
      801,
      1301,
      602,
      1192,
      711,
      1349
    ],
    "page": 1
  },
  {
    "_id": 1,
    "type": 0,
    "content": "https://www.homestuck.com/images/storyfiles/hs2/00002.gif",
    "thumbnail": "https://www.homestuck.com/images/storyfiles/hs2/00002.gif",
    "url": "https://homestuck.com/story/2",
    "tags": [
      1384,
      1385,
      391,
      321,
      4,
      1349,
      602
    ],
    "page": 2
  },
  ...
]
```
Our objective here is to combine the information so that, for each page, we'll have its human readable tags. 
"""


def load_hsse_tags(tags_path):
    """
    Loads and processes the HSSE tag definitions.

    Args:
        tags_path (str): Path to the hsse_tags.json file.

    Returns:
        dict: A dictionary mapping tag IDs (int) to tag names (str).
    """
    with open(tags_path, 'r', encoding='utf-8') as f:
        tags_data = json.load(f)

    # Extract the 'definitions' key
    definitions = tags_data.get('definitions', {})

    # Build mapping from _id (int) to name (str)
    tag_id_to_name = {}
    for tag_id_str, tag_info in definitions.items():
        tag_id = tag_info.get('_id')
        name = tag_info.get('name')
        if tag_id is not None and name is not None:
            tag_id_to_name[tag_id] = name
    return tag_id_to_name


def load_hsse_search(search_path):
    """
    Loads and processes the HSSE page-tag associations.

    Args:
        search_path (str): Path to the hsse_search.json file.

    Returns:
        dict: A dictionary mapping page IDs (str) to lists of tag IDs (int).
    """
    with open(search_path, 'r', encoding='utf-8') as f:
        search_data = json.load(f)

    # search_data is a list of dicts
    page_to_tag_ids = {}
    for entry in search_data:
        page = entry.get('page')  # integer
        tags = entry.get('tags', [])  # list of integers
        if page is not None:
            page_str = str(page)  # Convert page number to string to match 'pageId'
            page_to_tag_ids[page_str] = tags

    return page_to_tag_ids


def map_tags_to_names(page_to_tag_ids, tag_id_to_name):
    """
    Maps tag IDs to tag names for each page.

    Args:
        page_to_tag_ids (dict): Dictionary mapping page IDs to lists of tag IDs.
        tag_id_to_name (dict): Dictionary mapping tag IDs to tag names.

    Returns:
        dict: Dictionary mapping page IDs to lists of tag names.
    """
    page_to_tag_names = {}
    for page_id, tag_ids in page_to_tag_ids.items():
        # Ensure tag_ids is a list
        if not isinstance(tag_ids, list):
            tag_ids = [tag_ids]
        # Map each tag ID to its name, handle unknown tags gracefully
        tag_names = [tag_id_to_name.get(tag_id, f"Unknown Tag {tag_id}") for tag_id in tag_ids]
        page_to_tag_names[page_id] = tag_names
    return page_to_tag_names


def merge_tags_into_mspa_df(mspa_df, page_to_tag_names):
    """
    Merges HSSE tag data into the main MSPA DataFrame.

    Args:
        mspa_df (pd.DataFrame): The main MSPA DataFrame with transcripts.
        page_to_tag_names (dict): Dictionary mapping page IDs to lists of tag names.

    Returns:
        pd.DataFrame: The updated MSPA DataFrame with tags incorporated.
    """
    # Edit page_to_tag_names so the keys have the "6/" prefix to match the Viz page numbers
    page_to_tag_names = {f"6/{k}": v for k, v in page_to_tag_names.items()}

    # Create a Series from the page_to_tag_names dictionary
    tags_series = pd.Series(page_to_tag_names, name='tags')

    # Merge the tags into mspa_df, matching its vizPageId with the keys of the tags_series
    mspa_df_with_tags = mspa_df.merge(tags_series, left_on='vizPageId', right_index=True, how='left')

    # Replace NaN with empty lists for pages without tags
    mspa_df_with_tags['tags'] = mspa_df_with_tags['tags'].apply(lambda x: x if isinstance(x, list) else [])

    return mspa_df_with_tags


# Load HSSE tag definitions
tag_id_to_name = load_hsse_tags(HSSE_TAGS)
print(f"Loaded {len(tag_id_to_name)} tags from hsse_tags.json.")

# Load HSSE page-tag associations
page_to_tag_ids = load_hsse_search(HSSE_SEARCH)
print(f"Loaded tag associations for {len(page_to_tag_ids)} pages from hsse_search.json.")

# Map tag IDs to tag names
page_to_tag_names = map_tags_to_names(page_to_tag_ids, tag_id_to_name)
print("Mapped tag IDs to tag names.")

# Merge tags into the main MSPA DataFrame
mspa_df_with_tags = merge_tags_into_mspa_df(mspa_panels_annotated_df, page_to_tag_names)

# # Display the first few rows with tags not an empty list
mspa_df_with_tags[mspa_df_with_tags['tags'].apply(len) > 0].sample(10)

Loaded 1427 tags from hsse_tags.json.
Loaded tag associations for 8002 pages from hsse_search.json.
Mapped tag IDs to tag names.


,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript,tags
2560,story,002081,TT: Deploy Pre-punched Card.,,1242785006,/storyfiles/hs2/00181_2.gif,[002082],002080,,[],Homestuck,181,6/181,https://homestuck.com/story/181,6/002081,NaN,NaN,NaN,NaN,"[Act 1, Part 1, John Egbert, John's House, Hou..."
4529,story,003702,Rose: Alchemize a whole bunch of cool stuff.,"<a href=""http://www.mspaintadventures.com/swee...",1273121501,/storyfiles/hs2/01802.gif,[003703],003701,,[],Homestuck,1802,6/1802,https://homestuck.com/story/1802,6/003702,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Rose Lalonde, Consort, Alchemi..."
3674,story,003002,AR: Examine the wall behind you.,The wall exhibits rows of ancient hieroglyphs ...,1262480882,/storyfiles/hs2/01102_2_retcon.gif,[003003],003001,,[R4],Homestuck,1102,6/1102,https://homestuck.com/story/1102,6/003002,NaN,NaN,NaN,NaN,"[Part 1, Act 3, Frog Temple, Skaianet Logo, Frog]"
3342,story,002749,==>,,1257834304,/storyfiles/hs2/00849.gif,[002750],002748,,[],Homestuck,849,6/849,https://homestuck.com/story/849,6/002749,NaN,NaN,NaN,NaN,"[Part 1, Act 3, Rose Lalonde, Skaianet Lab]"
4684,story,003796,==>,,1274420542,/storyfiles/hs2/01896.gif,[003797],003795,,[],Homestuck,1896,6/1896,https://homestuck.com/story/1896,6/003796,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Incipisphere, Grandpa, Meteor]"
2758,story,002249,Dave: Clean up the juice.,"You <span style=""color: blue"">CL</span><span s...",1247014616,/storyfiles/hs2/00349_2.gif,[002250],002248,,[],Homestuck,349,6/349,https://homestuck.com/story/349,6/002249,NaN,NaN,NaN,NaN,"[Act 2, Part 1, Dave Strider, Record Shirt, Da..."
4065,story,003313,==>,But instead she takes note of your nice chalk ...,1266530092,/storyfiles/hs2/01413_1.gif,[003314],003312,,[],Homestuck,1413,6/1413,https://homestuck.com/story/1413,6/003313,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Carapacian, Exile, PM, Oil, Dr..."
2571,story,002089,TT: Explore Atheneum.,Acquiring a CRUXITE DOWEL seems to have popula...,1242943802,/storyfiles/hs2/00189.gif,[002090],002088,,[],Homestuck,189,6/189,https://homestuck.com/story/189,6/002089,NaN,NaN,NaN,\nPerfectly Generic Object\n2\n,"[Act 1, Part 1, Sburb, Grist]"
3092,story,002537,John: Attack the nearest imp to test pogo hamm...,You get a vicious rhythmic bouncing combo goin...,1252908310,/storyfiles/hs2/00637.gif,[002538],002536,,[],Homestuck,637,6/637,https://homestuck.com/story/637,6/002537,NaN,NaN,NaN,BOING,"[Part 1, Act 2, John's House, John Egbert, Gre..."
2372,story,001927,John: Look out window.,You see the view of your yard from your window...,1239831633,/storyfiles/hs2/00027_1.gif,[001928],001926,,[],Homestuck,27,6/27,https://homestuck.com/story/27,6/001927,NaN,NaN,NaN,NaN,"[Part 1, Act 1, Earth, John's House]"


# Extracting character appearances from POV cam 

The POV cam extension for Homestuck allows us to see the characters that are present in each page, and not just until page 1988, all of them. We can use this to extract character tags and somewhat make up for the lack of tags in the later pages.

In [58]:
"""
The data from the POV cam comes in many files named after each character, like "roxy.txt" and "rufioh.txt". The format is not meant to be easily parsable, but it shouldn't be too hard to extract the data and "invert" it, to get the characters that appear in each page and their "commands".

An example of the data (jade.txt):
```
Name: Jade
Colour: #4AC925
Image: jade.png
Group: Kids

Be created on meteor
3790-3791
3803
3807
3830-3831

Be sent to Earth
3840

Land on factory
3768-3769

Be adopted
3773-3775

Be taken on hunt with grandfather
Wander off with Bec
Find present
3029-3036
```
From the documentation:
```md
## Timeline language

In the `Readable Timelines` folder are a number of files, each containing the timeline data for a single person.

The files use the following format:

 * Page numbers or ranges of numbers to describe what pages a person's on.
   (For A6A5A1x2 COMBO, use `-2` on the end to go through the pages two at a time)
   eg. `6009`, `1901-2032`, or `7688-7692-2`
 * To split the timeline, indent the splintered timeline, then return to the original indentation for the alpha timeline.
   Note that the two timelines are not connected by default, you must use the next two commands to describe how they should be joined.
 * `==>`: Jump into the next split timeline from this point
 * `<==`: Jump out of previous split timeline to this point
 * `~`: Insert another timeline here, using a person's name.
   eg. `~ Davesprite`

The following commands change properties about the current person or timeline.
Write the exact word, then `:`, then the value you wish to set it to.
eg. `Name: John`.

 * `Name`: Change the name of the current person.
 * `Colour`: Change the colour used for the links.
 * `Image`: Change the image used for the links.
 * `Group`: Change which group the links are a part of.
 * `Caption`: Give some hover-over text to the link between the previous page and the next.

All lines which do not fit any of the above are ignored, like comments.
```
"""


def extract_data_from_pov_cam_file(pov_cam_file: str) -> pd.DataFrame:
    """
    Extracts data from a POV cam file.

    Args:
        pov_cam_file (str): The path to the POV cam file.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted data with columns ['page', 'character', 'commands'].
    """
    # Regular expression to identify page lines (single number or range with optional step)
    page_pattern = re.compile(r'^\d+(-\d+)?(-\d+)?$')

    try:
        with open(pov_cam_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()
    except FileNotFoundError:
        print(f"Error: File not found - {pov_cam_file}")
        return pd.DataFrame(columns=['page', 'character', 'commands'])
    except Exception as e:
        print(f"Error reading file {pov_cam_file}: {e}")
        return pd.DataFrame(columns=['page', 'character', 'commands'])

    # Initialize the data storage
    data = {
        'page': [],
        'character': [],
        'commands': []
    }

    # Initialize current character and commands
    current_character = None
    current_commands = []
    last_commands = []

    for line_number, line in enumerate(lines, start=1):
        original_line = line  # Keep the original line for debugging if needed
        line = line.strip()

        if not line:
            continue  # Skip empty lines

        # Check for metadata lines
        if line.startswith("Name: "):
            # Extract character name from the 'Name:' line
            extracted_name = line.split("Name: ", 1)[1].strip()
            if current_character is None:
                current_character = extracted_name
            else:
                # Ignore subsequent 'Name:' lines to prevent name changes from affecting the character name
                continue
            current_commands = []  # Reset commands when a new character is found
            last_commands = []  # Reset last_commands for the new character
            continue  # Move to the next line
        elif any(line.startswith(prefix) for prefix in ["Colour: ", "Image: ", "Group: "]):
            continue  # Skip other metadata lines

        # Check if the line is a page line
        if page_pattern.match(line):
            if current_commands:
                # If there are new commands, update last_commands
                last_commands = current_commands.copy()
                commands_to_use = current_commands.copy()
            elif last_commands:
                # If no new commands, inherit last_commands
                commands_to_use = last_commands.copy()
            else:
                # If no commands to inherit, assign ["==>"]
                commands_to_use = ["==>"]

            # Parse the page or range
            pages = []
            parts = line.split('-')
            try:
                if len(parts) == 1:
                    pages = [int(parts[0])]
                elif len(parts) == 2:
                    start, end = map(int, parts)
                    pages = list(range(start, end + 1))
                elif len(parts) == 3:
                    start, end, step = map(int, parts)
                    pages = list(range(start, end + 1, step))
                else:
                    print(f"Warning: Unexpected page format at line {line_number}: '{original_line}'")
                    continue  # Skip unexpected formats
            except ValueError:
                print(f"Error: Non-integer page numbers at line {line_number}: '{original_line}'")
                continue  # Skip lines with non-integer values

            for page in pages:
                if page in data['page']:
                    # If the page already exists, append the new commands without duplication
                    index = data['page'].index(page)
                    existing_commands = data['commands'][index]
                    for cmd in commands_to_use:
                        if cmd not in existing_commands:
                            existing_commands.append(cmd)
                else:
                    data['page'].append(page)
                    data['character'].append(current_character)
                    data['commands'].append(commands_to_use.copy())  # Use a copy to prevent mutation

            current_commands = []  # Reset commands after assigning them to pages
        else:
            # Assume it's a command line
            current_commands.append(line)

    # Remove duplicate commands per page while preserving order
    for i, cmds in enumerate(data['commands']):
        seen = set()
        unique_cmds = []
        for cmd in cmds:
            if cmd not in seen:
                unique_cmds.append(cmd)
                seen.add(cmd)
        data['commands'][i] = unique_cmds

    # Create DataFrame
    df = pd.DataFrame(data)
    return df


def extract_pov_cam_data(pov_cam_folder: str) -> pd.DataFrame:
    """
    Extracts data from all POV cam files in a given folder.

    Args:
        pov_cam_folder (str): The path to the folder containing POV cam text files.

    Returns:
        pd.DataFrame: A combined DataFrame containing data from all POV cam files.
    """
    # List all .txt files in the folder
    pov_cam_files = [file for file in os.listdir(pov_cam_folder) if file.endswith('.txt')]
    pov_cam_file_with_full_path = [os.path.join(pov_cam_folder, file) for file in pov_cam_files]

    # Initialize a list to hold DataFrames from each file
    pov_cam_data_frames = []

    for file_path in pov_cam_file_with_full_path:
        df = extract_data_from_pov_cam_file(file_path)
        if not df.empty:
            pov_cam_data_frames.append(df)
        else:
            print(f"No data extracted from {file_path}")

    if pov_cam_data_frames:
        pov_cam_data = pd.concat(pov_cam_data_frames, ignore_index=True)
    else:
        pov_cam_data = pd.DataFrame(columns=['page', 'character', 'commands'])

    return pov_cam_data


# Extract data from the POV cam files
pov_cam_data = extract_pov_cam_data(POV_CAM_FOLDER)

# Display the extracted data
pov_cam_data.sample(10)

,page,character,commands
13148,3830,Rose,[Be created]
11713,6885,Karkat,[Enter dream bubble]
10682,4604,Kanaya,"[View Rose, Troll John]"
17718,9752,Vriska,"[Call Jake, Request debriefing from Meenah, Fa..."
12455,6896,Meulin,"[Fail Sgrub, Witness initiation of Scratch, Be..."
9900,8070,John,"[Jump around space, time]"
4048,5535,Doc Scratch,[Talk with Rose]
5838,8625,Jack Noir,[Rescue Jade]
2971,9647,Dave,"[<====, Confirm which version of Jack Noir you..."
9472,5364,John,"[Crash car, Talk with Vriska about killing Tav..."


In [59]:
# Show some unique complex data, like pages with multiple commands
pov_cam_data[pov_cam_data['commands'].apply(len) > 2].sample(10)

,page,character,commands
10732,5218,Kanaya,"[View Rose when viewport goes dark, Talk to Ja..."
7285,7655,Jake,"[Be found by trickster Jane, Enter trickster m..."
14625,8198,Roxy,"[Be given mission by Jade, Be intimidated by J..."
17201,8030,Trace,"[Be resurrected by SS, Be commanded by SS, Lea..."
10725,5211,Kanaya,"[View Rose when viewport goes dark, Talk to Ja..."
9933,8105,John,"[Staredown with Caliborn, Get punched into nex..."
8990,2535,John,"[Shuffle in captchalogue cards, Make pogo hamm..."
9432,4982,John,"[Level up., Enter village., Proceed to Quest B..."
10758,5245,Kanaya,"[View Rose when viewport goes dark, Talk to Ja..."
2053,8031,Clover,"[Be resurrected by SS, Be commanded by SS, Lea..."


In [60]:
# Now that we have the page/character/commands dataset, we can merge the page/character data
# We'll turn the commands into an array of commands prefixed with the character name
# We'll also add a "pov_characters" and "pov_commands" column to the mspa_df
# Note that for the POV cam, the vizPageId is actually incorrect. We can use the actual page number in pageId instead,
# though it's six padded digits in mspa_df_with_tags and an actual integer in pov_cam_data. We'll padd the latter to match.

# Pad the page numbers in pov_cam_data
pov_cam_data['pageId'] = pov_cam_data['page'].apply(lambda x: f"{x:06d}")

# Merging the data now would be silly, we need to create a new dataframe where there's only one row per page
# We'll concatenate the commands for each character in the same page
# We'll also add the commands to a list, and the characters to a list

# Add the character names to the commands if the commands don't already have them
pov_cam_data['commands'] = pov_cam_data.apply(lambda x: [f"{x['character']}: {cmd}" for cmd in x['commands']], axis=1)

# Group by pageId and character
grouped = pov_cam_data.groupby(['pageId']).agg({
    'character': list,
    'commands': list
}).reset_index()

grouped.sample(10)

,pageId,character,commands
7349,009575,"[Dave, Dirk]","[[Dave: <====, Dave: Have awkward conversation..."
1605,003517,"[Parcel Mistress, White Queen]","[[Parcel Mistress: Seek audience with WQ], [Wh..."
379,002285,"[John, Warweary Villein]","[[John: STAY WARY OF THESE FOES.], [Warweary V..."
4052,006107,[Jane],"[[Jane: Sneak out of house, Jane: Talk with Ja..."
4450,006521,[Roxy],[[Roxy: Be snuck up upon by Dersites]]
21,001922,[John],"[[John: Be introduced, John: Mess around]]"
1667,003593,"[Dave, John, Rose]",[[Dave: Convince John to rummage through Rose'...
748,002654,[Warweary Villein],"[[Warweary Villein: WV: Retri..., Ascend.]]"
6848,009064,"[Meenah, Vriska]","[[Meenah: Enter Dark Carnival with Vriska], [V..."
1098,003006,[Authority Regulator],[[Authority Regulator: Seek more powerful weap...


In [61]:
# Merge the data into mspa_df_with_tags
# "character" can become "pov_characters" and "commands" can become "pov_commands"

# rename columns
grouped.rename(columns={'character': 'pov_characters', 'commands': 'pov_commands'}, inplace=True)

mspa_df_with_pov = mspa_df_with_tags.merge(grouped, on='pageId', how='left')

# Replace NaN with empty lists for pages without POV cam data
mspa_df_with_pov['pov_characters'] = mspa_df_with_pov['pov_characters'].apply(
    lambda x: x if isinstance(x, list) else [])
mspa_df_with_pov['pov_commands'] = mspa_df_with_pov['pov_commands'].apply(lambda x: x if isinstance(x, list) else [])

mspa_df_with_pov[mspa_df_with_pov['pov_characters'].apply(len) > 0].sample(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands
10967,story,009508,[A6A6I5] ====>,,1433112919,/storyfiles/hs2/07604.gif,[009509],009507,,[],...,6/7608,https://homestuck.com/story/7608,6/009508,NaN,NaN,NaN,NaN,[],[Calliope],"[[Calliope: Meet Calliope, Calliope: Talk with..."
4282,story,003488,Rose: Answer Dave.,"|PESTERLOG|<br /><span style=""color: #e00707"">...",1268351760,/storyfiles/hs2/01588_1.gif,[003489],003487,,[],...,6/1588,https://homestuck.com/story/1588,6/003488,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Dave's Room, Dave's House, Dav...","[Dave, Rose]","[[Dave: Talk to Rose, Kanaya], [Rose: Strife, ..."
4579,story,003733,Dave: Combine batarang + Midnight Crew poster.,You make a whole pile of SUITARANGS because th...,1273786994,/storyfiles/hs2/01833_1.gif,[003734],003732,,[],...,6/1833,https://homestuck.com/story/1833,6/003733,NaN,NaN,NaN,\nSUITARANGS\n⬡ 20 ▲ 4\n,"[Part 1, Act 4, Dave Strider, Suit, Red Plush ...",[Dave],[[Dave: Wake up and jam.]]
8478,story,007178,[A6I3] ==>,,1347150293,/storyfiles/hs2/05275.gif,[007179],007177,,[],...,6/5278,https://homestuck.com/story/5278,6/007178,NaN,NaN,NaN,NaN,[],"[Jade, John]","[[Jade: Year 2, Jade: Watch Con Air with John]..."
4717,story,003821,==>,Ah ha! There's HALLEY. The youngsters adore th...,1274670991,/storyfiles/hs2/01921.gif,[003822],003820,,[],...,6/1921,https://homestuck.com/story/1921,6/003821,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Nanna, Grandpa, Weapon, Gun, F...","[Authority Regulator, Grandpa, Nanna]","[[Authority Regulator: Explore Frog Temple], [..."
5652,story,004618,==>,Uh...,1285908380,/storyfiles/hs2/02717_2.gif,[004619],004617,,[],...,6/2718,https://homestuck.com/story/2718,6/004618,NaN,NaN,NaN,NaN,[],"[Kanaya, Rose]","[[Kanaya: Hop to 8=8.], [Rose: Blow up your fi..."
8521,story,007220,[A6I3] ==>,,1348706750,/storyfiles/hs2/05317_retcon.gif,[007221],007219,,[R4],...,6/5320,https://homestuck.com/story/5320,6/007220,NaN,NaN,NaN,NaN,[],"[Jack Noir, Parcel Mistress]","[[Jack Noir: Be chased by PM], [Parcel Mistres..."
6779,story,005643,Insert disc 2.,,1303945970,/storyfiles/hs2/03740/03740.swf,[005644],005642,,"[F, SWFNAV]",...,6/3743,https://homestuck.com/story/3743,6/005643,NaN,NaN,NaN,NaN,[],[Terezi],"[[Terezi: Scratch disc 2, Terezi: Ascend.]]"
7670,story,006418,Dirk: Go get your sword.,You retrieve your UNBREAKABLE KATANA. A real h...,1327976942,/storyfiles/hs2/04515.gif,[006419],006417,,[],...,6/4518,https://homestuck.com/story/4518,6/006418,NaN,NaN,NaN,NaN,[],"[Dirk, Lil' Cal]","[[Dirk: Be introduced, Dirk: Rap off with Squa..."
9281,story,007874,[A6I5] ==>,"|DIALOGLOG|<br /><span style=""color: #005682"">...",1362869446,/storyfiles/hs2/05971.gif,[007875],007873,,[],...,6/5974,https://homestuck.com/story/5974,6/007874,NaN,NaN,NaN,NaN,[],"[Aradia, Aranea, Caliborn, Calliope, Feferi, J...","[[Aradia: Be in Vriska's crew], [Aranea: Join ..."


# Adding Acts and Pages of Homestuck data

I completely forgot this existed until I was already trying to finetune a model, but this is critical information. For each act and even groups of pages, we have detailed synopses. We'll actually need to crawl and generate the dataset ourselves.

In [62]:
"""
The target is https://rafe.name/homestuck.
We'll obviously have to crawl all the act pages, and the part pages. Each part contains multiple acts, each act contains multiple groups of pages. 
We'll end up with a dataset like...
page_id | page_group_summary | act | act_summary | part | part_summary
1 | "John Egbert examines his room." | "Act 1" | [["JOHN", "Homestuck begins as the story of four friends who live far apart and have never met. We are introduced to these kids one by one, starting with John Egbert...."], ["SBURB", "The alchemy equipment emits a confusing ball of light called the Kernelsprite. The sprite is “prototyped” when it absorbs John’s maimed harlequin doll and takes on the appearance of a one-eyed, one-armed jester..."]] | "Part 1" | "John, Rose, Dave, and Jade are online friends. On John’s thirteenth birthday, the kids play a game that takes them and their guardians to a set of fantastic worlds called the Medium. They are challenged to work together, kill monsters, learn punch-card alchemy, and build their houses up to the heavenly orb of Skaia..."

This should be really useful when we're trying to guess the tags that are in a page. The summaries should also be generally useful for text-based models later on. 
(Note: obviously I ended up making three datasets instead because the act/part transcripts are HUUUUUUUUUGE.)
"""

ACTS_AND_PAGES_URL = "https://rafe.name/homestuck/"

# Once we have the file in the repo, no need to crawl again
FORCE_ACTS_CRAWL = False

# Rafe never made a summary of part 4, so I made one myself
PART_FOUR_SUMMARY = """Caliborn seizes control of the narrative. He replaces Homestuck with his own twisted version, "Homosuck," mocking the story and its creator. Caliborn's aim is to undermine and ridicule everything, from the protagonist's struggles to the art itself. Despite his effort to hijack the narrative, Caliborn unwittingly contributes to the story's continuation, as items left by The Author lead him to unlock new content, extending Homestuck even as he tries to derail it.

In the main narrative, the players have just arrived to the new session. The scattered kids face challenges under the Condesce's plan to take over their session and its nascent Universe, with Jade and Jane falling under her control, and Jake, Roxy and Dirk imprisoned or otherwise AWOL. 

Aranea uses the Ring of Life juju to come back to life. After a massive battle, where most characters die due to the Condesce and Aranea's meddling, John meets his Denizen in desperation, and gains a power to jump across the canon of the narrative itself. Directed by a dying Terezi's instructions, John "retcons" events so that Vriska survives instead of dying. Vriska takes an active role in helping the group strategize for the upcoming showdown.

In the final ACT ACT ACT, Caliborn shows a "spoiler to the ending of things" in Claymation form. In his tale, eight kids show up to defeat him, and four of them (which seem to be some version of John, Rose, Dave and Jade) are trapped inside a house-shaped juju. The four remaining combatants, implied to be the Alpha Kids, cause the ultimate defeat of Caliborn, but also the creation of Lord English. The ultimate villain is revealed to be a product of the combined essences of Caliborn, Arquiusprite, and Gamzee, created as an unwitting side effect of one Dirk's attack. A combination of their souls is trapped inside the Lil Cal juju, which is (only temporally, as we've previously seen) sent into the Void by one Roxy.

In the present time and session, and with proper planning, an even more intense battle ensues. It involves all of the main players, Spades Slick, Lord Jack, and The Condesce. Roxy manages to land a killing blow on The Condesce, and Dave and Dirk defeat the Jacks. Avoiding all casualties, the players reunite at the victory lilypad, where Calliope is resurrected, finally able to take her place in the waking world.

After a long sequence of post-battle conversations, the players use the power of the Forge and the Genesis Frog create a new universe, Universe C. In the Furthest Ring, the alternate Calliope, the Muse of Space, helps collapse the Green Sun into a Black Hole, where Vriska deploys the secret weapon against Lord English, a house-shaped juju, but his defeat is not shown.

The surviving players and their allies pass through the door to claim the Ultimate Reward: a new world for themselves, where the previous Earth B (now C) is placed and prepared to support new life. The players are crowned as the creators of this new universe, and they begin to rebuild civilization. The story concludes with a hint that John still needs to return to finish the job and defeat Caliborn, a closed loop that will end with Lord English's creation."""

# Taken from page descriptions, better than nothing
A6A6A6_SUMMARY = """Jane and the Nannas are ready to heal. Jasprose’s windows will move the healers between the fights in the session. Jane can resurrect each fighter once.
Davepeta, Aradia, and Sollux find the ghost army. Karkat moves first, charging fearlessly into Lord English’s double death beam. Karkat wakes up on Lofaf.
Strife begins on Derse. Rose, John, and Roxy play fraymotifs to clobber Condy with a Zazzerpan statue, whip her with wind, and pummel her with perfectly generic objects. Condy fights back with all the psionic powers that will avail her.
Strife begins on Lomax. Itchy and Stitch get the first hits in. Jake shoots Sawbuck and time-travels to several other moments in the fight. Arquiusprite joins, taunting Cans with quadrupedal automata and the fridge in which Gamzee is still locked. Cans clocks Jake, Arquius, and several Felt to Lofaf, where Clover humiliates Karkat, and again back to Lomax. Crowbar shoots and kills Jake.
The healing squad goes first to Derse, where Rose and Kanaya have died. Jane resurrects them while Jasprose distracts Condy. The fight on Derse reaches its most chaotic and madcap when Cans clocks Arquius and the fridge into it. Next, the healers go to Lomax and resurrect Jake. Everyone loves Nanna’s cookies. Jane stays to fight aside Jake. Casey von Salamancer’s skeleton army overruns the combatants.
Strife begins on Lotak, first between the Jacks. The three sides fight indiscriminately. Dave and Terezi play a fraymotif to anticipate and avert bad outcomes. Dirk tries to rip Lord Jack’s soul, but the Archagent is already too ripped with clockwork majyyks. Cans clocks Arquius and the fridge into their fight, then clocks Arquius back to Lofaf, leaving the fridge on Lotak.
Arquius and Cans end up on Derse in time to see Dad Crocker break out of jail. Dad pies Cans.
Strife begins in the ruin of Prospit between the chess dogs and in the bubble between Vriska and the Author.
Caliborn defeats Yaldabaoth and finds his reward: a judgment clock and a chest containing the weapon he will use in his masterpiece.
The Muse of Space reaches the Green Sun.
Serenity finds John on Derse, Jade on Prospit, Dave on Lotak, and Karkat on Lofaf, filling them with resolve. Finally, Serenity is reunited with the Vagabond.
Strife begins in the bubble between the ghost army and the constant conqueror. Lord English annihilates many ghosts. Meenah, Tavros, Aradia, and Davepeta put up a fight.
John, Roxy, Rose, and Kanaya regroup against Condy, who has choked Jane. Dave, Dirk, and Terezi renew the fight against the Jacks. They all take fearsome beatings, but they have come so far that they will not give up now.
Gamzee has been rattled around in the Crocker family fridge to the point of tears.
Jake, Karkat, and Dad Crocker vanquish The Felt.
To stop the chess dogs fighting, Jade teleports them from place to place and puts herself between them. They still have Bec’s loyalty and love of her. This may keep them from stabbing her, but not from using non-lethal methods. The Monarch punches Jade out and resumes her quest to neutralize the Slayer.
The Archagent gets Dirk in a hold with his crowbar. Spades Slick gets the Archagent in a hold with his cuestaff. Their heads are all lined up, so Dave takes the shot. He decapitates them and breaks the unbreakable katana in one blow. Then, for the first time since his old session, Dave uses time travel. He recovers Dirk’s head and body and evacuates Terezi from Lotak. Their enemies may be dead, but the Archagent’s cursed clockwork head is a bomb. It implodes into a black hole, taking the fridge and most of Lotak with it.
The Monarch cuts off the Slayer’s arm. He loses his ring and is Bec Noir no more. He gets the non-lethal methods too. He lands unconscious next to Jade.
The Batterwitch makes the fatal mistake of turning her back on Roxy, who impales her on Bro’s sword. Ding dong.
Nanna heals Jane and meets Dad Crocker. The Derse and Lotak teams return to the lilypad. Jane resurrects Dirk.
Jade wakes. Callie has made a new friend, and Jack Noir perhaps a new sweetheart, in Ms. Paint.
The Author, done hassling Vriska, gives Lord English the big thumbs down.
The Monarch finds the Vagabond and takes off her Ring of Orbs Fourfold. She wants both rings to be unmade in the Forge.
Jake, Karkat, Jade, and Callie come to the lilypad. They have joyful reunions all round.
Dad Crocker meets John, whose post-scratch self was Dad’s father.
The Monarch and Vagabond climb to the lip of the Forge."""

# Taken from page descriptions, better than nothing
ACT_7_SUMMARY = """The Rings of Orbs Fourfold light the Forge, propelling the genesis tadpole into Skaia. The tadpole begins to metamorphose and dives into the hole in the Battlefield by which John excavated The Tumor. There, the tadpole completes its metamorphosis. It eats the Battlefield and becomes the splendid Genesis Frog, embodiment of a new universe.
The Muse of Space makes a black hole next to the Green Sun. It eats the Sun and the surrounding space of the Furthest Ring. Aradia gets her wish to see this whole place break apart.
Vriska is ready to use her weapon. She opens the chest and produces an image of a house which contains the aspects of Breath, Light, Time, and Space. This artefact does something ambiguous to Lord English.
In Yaldabaoth’s lair, Caliborn smashes the judgment clock and receives the clockwork majyyks.
A door to the ultimate reward appears before those assembled on the lilypad. They will take Earth through to their new universe and terraform it. The ocean which covered Earth will recede and terrestrial life will return. Carapacians will live on Earth with the victors, fulfilling the promise of Can Town.
John reaches for the doorknob.
In snapshots from Earth C, the victors return to the lab meteor which crashed into Earth. They go to the ectobiology lab and make human, troll, and carapacian babies. They leave these babies in the care of mature carapacians, then time travel forward to live in a later stage of the resulting civilization.
They are hailed as the “creators” and proclaimed kings and queens:
John and Jane are crowned in the Human Kingdom, accompanied by Dad and Nannasprite.
Karkat, Terezi, Dave, and Jade are crowned in the Troll Kingdom.
Rose, Roxy, Kanaya, and Callie are crowned in the Carapacian Kingdom.
Jake and Dirk are crowned in the Consort Kingdom, accompanied by GCatavrosprite.
John celebrates his seventeenth birthday. Rose and Kanaya marry. Dave, Karkat, and Jade excavate shitty Liberties. Dirk makes a skull-monster–shaped sparring partner.
By the time John turns eighteen, Terezi has left Universe C to look for Vriska in the disintegrating Furthest Ring. Her search seems fruitless.
Jane remakes CrockerCorp. Jake remakes SkaiaNet, though its only seeming function is to get his ass on television. Kanaya oversees the Mother Grub and her wiggler broods.
On his nineteenth and twentieth birthdays, John is increasingly solitary. Finally Caliborn sends him a snap spoiling for a fight. This may be what provokes the battle depicted in Caliborn’s masterpiece."""

part_urls = ["part-1", "part-2", "part-3", "part-4"]

act_urls_per_part = {
    "part-1": ['act-1', 'act-2', 'act-3', 'intermission', 'act-4'],
    "part-2": ['a5a1', 'a5a2', 'eoa5'],
    "part-3": ['a6-1', 'a6-2', 'a6-3', 'a6-4', 'a6-5'],
    "part-4": ['a6a6-1', 'a6a6-2', 'a6a6-3', 'a6a6-4', 'a6a6-5', 'a6a6a6', 'act-7']
}


def _clean_cell_html(cell: str) -> str:
    # Acts and pages has the ugly habit of prefixing and affixing each cell with a bunch of \n linebreaks
    if not cell:
        return cell
    # Trim multiple \ns from the start
    left_strip = cell.lstrip('\n')
    # Trim multiple \ns from the end
    right_strip = left_strip.rstrip('\n')
    return right_strip


part_summaries = {}
act_summaries = {}


def crawl_acts_and_pages(url: str) -> tuple[dict, dict, list]:
    acts_and_pages = []
    part_summaries = {}
    act_summaries = {}

    for part_url in part_urls:
        # Crawl part description, located in <table class="part">, first row, first column
        part_link = f"{url}{part_url}"
        part_response = requests.get(part_link)
        part_soup = BeautifulSoup(part_response.content, 'html.parser')

        part_table = part_soup.find('table', class_='part')
        if part_table is None:
            continue
        part_td_elements = part_table.find_all('td')

        if len(part_td_elements) < 1:
            continue

        part_summary = part_td_elements[0].get_text().strip() if part_url != 'part-4' else PART_FOUR_SUMMARY
        part_summaries[part_url] = part_summary

        for act in act_urls_per_part[part_url]:
            # Crawl act synopses, there are multiple synopses per act, organized per viewpoint or theme
            # Each is stored in one <tr class="synopsis">, with the first <th> being the viewpoint/theme,
            # we will make it allcaps before storing it in an array in the right key of the act_summaries dict
            act_link = f"{url}{act}"
            act_response = requests.get(act_link)
            act_soup = BeautifulSoup(act_response.content, 'html.parser')
            act_synopses = act_soup.find_all('tr', class_='synopsis')
            if act == 'eoa5':
                # rafe erroneously called eoa5/intermission 2 a different act, it should just be a5a2
                # Grab it from a5a2, which should be processed right before this
                act_summary = act_summaries['a5a2']
            elif act == 'a6a6a6':
                act_summary = [["GENERAL", A6A6A6_SUMMARY]]
            elif act == 'act-7':
                act_summary = [["GENERAL", ACT_7_SUMMARY]]
            else:
                act_summary = []
                for synopsis in act_synopses:
                    viewpoint = synopsis.find('th').get_text().upper()
                    if not viewpoint:
                        viewpoint = "GENERAL"
                    summary = synopsis.find('td').get_text().strip()
                    act_summary.append([viewpoint, summary])
            act_summaries[act] = act_summary

            # And now we crawl the page data in the act pages
            # Iterate over all tables that are not the sidebar or header navigation
            page_data_tables = [
                table for table in act_soup.find_all('table')
                if table.get('id') not in ['sidebar-nav', 'header-nav']
            ]

            for page_data_table in page_data_tables:
                page_data_rows = page_data_table.find_all('tr')
                for row_index, row in enumerate(page_data_rows):
                    # get rows that have at least four <td> elements or contain a table with class x2
                    cells = row.find_all('td')
                    if len(cells) < 4 and not row.find('table', class_='x2'):
                        continue
                    page_links = cells[-1].find_all('a')
                    # extract the PAGE_NUMBER from the a hrefs
                    page_numbers = [link['href'].split('/')[-1] for link in page_links]
                    # extract the HTML summary from the third column
                    page_summary = cells[2].get_text().strip()
                    # Handle x2 table data if present
                    x2_table = cells[2].find('table', class_='x2')
                    if x2_table:
                        # Remove the x2 table from the original summary
                        x2_table.extract()
                        page_summary = cells[2].get_text().strip()
                        x2_rows = x2_table.find_all('tr')
                        for x2_row in x2_rows:
                            x2_cells = x2_row.find_all('td')
                            if len(x2_cells) == 2:
                                x2_content = f" {x2_cells[0].get_text().strip()} | {x2_cells[1].get_text().strip()}"
                                page_summary += x2_content
                    # save everything per page
                    if act == 'eoa5':
                        act = 'a5a2'
                    for page_number in page_numbers:
                        acts_and_pages.append({
                            'page_id': page_number,
                            'page_group_summary': page_summary,
                            'act': act,
                            'part': part_url
                        })

    return part_summaries, act_summaries, acts_and_pages


# Let's check if we already have the files for parts, acts, and pages
if os.path.exists(ACTS_AND_PAGES_PART_SUMMARIES) and os.path.exists(ACTS_AND_PAGES_ACT_SUMMARIES) and os.path.exists(
        ACTS_AND_PAGES_PAGE_SUMMARIES) and not FORCE_ACTS_CRAWL:
    with open(ACTS_AND_PAGES_PART_SUMMARIES, 'r', encoding='utf-8') as f:
        part_summaries = json.load(f)
    with open(ACTS_AND_PAGES_ACT_SUMMARIES, 'r', encoding='utf-8') as f:
        act_summaries = json.load(f)
    with open(ACTS_AND_PAGES_PAGE_SUMMARIES, 'r', encoding='utf-8') as f:
        page_summaries = json.load(f)

    print("Acts and Pages data loaded from files.")

else:
    print("Acts and Pages data not found. Crawling website...")

    # Crawl the website to extract the Acts and Pages data
    part_summaries, act_summaries, page_summaries = crawl_acts_and_pages(ACTS_AND_PAGES_URL)

    # Save the extracted data to JSON files
    with open(ACTS_AND_PAGES_PART_SUMMARIES, 'w', encoding='utf-8') as f:
        json.dump(part_summaries, f)
    with open(ACTS_AND_PAGES_ACT_SUMMARIES, 'w', encoding='utf-8') as f:
        json.dump(act_summaries, f)
    with open(ACTS_AND_PAGES_PAGE_SUMMARIES, 'w', encoding='utf-8') as f:
        json.dump(page_summaries, f)

    print("Acts and Pages data saved to files.")

Acts and Pages data loaded from files.


In [63]:
part_summaries_df = pd.DataFrame(part_summaries.items(), columns=['part', 'part_summary'])
part_summaries_df

,part,part_summary
0,part-1,"John, Rose, Dave, and Jade are online friends...."
1,part-2,"Aradia, Tavros, Sollux, Karkat, Nepeta, Kanaya..."
2,part-3,"In the first session, Skaia sent the cloned ba..."
3,part-4,Caliborn seizes control of the narrative. He r...


In [64]:
act_summaries_df = pd.DataFrame(act_summaries.items(), columns=['act', 'act_summary'])
act_summaries_df

,act,act_summary
0,act-1,"[[JOHN, Homestuck begins as the story of four ..."
1,act-2,"[[THE MEDIUM, John narrowly escapes fiery deat..."
2,act-3,"[[JADE, Our fourth kid is the sleepy Jade Harl..."
3,intermission,"[[THE MANSION, Spades Slick, Diamonds Droog, C..."
4,act-4,"[[KIDS, Previously: John reached the First Gat..."
5,a5a1,"[[SGRUB, Previously, the aliens that were trol..."
6,a5a2,"[[GENERAL, This synopsis covers Act 5 Act 2, C..."
7,eoa5,"[[GENERAL, This synopsis covers Act 5 Act 2, C..."
8,a6-1,"[[JANE, Act 6 begins on Earth on the flip side..."
9,a6-2,"[[TWO DOWN, After Jane gets knocked out at the..."


In [65]:
page_summaries_df = pd.DataFrame(page_summaries)
page_summaries_df.sample(5)

,page_id,page_group_summary,act,part
7122,7122,John talks about how he fought Caliborn. Roxy ...,a6a6-4,part-4
6274,6274,Act 6 Act 6 Act 1.\nCinematic 97: Homosuck.\n ...,a6a6-1,part-4
6998,6998,John and Roxy grieve and contemplate giving up...,a6a6-4,part-4
5348,5348,"Exploiting Rose’s ruse, John bonks Noir.",a6-3,part-3
2431,2432,Feferi Peixes feeds Gl’bgolyb.,a5a1,part-2


In [66]:
# Let's check every page number and see if we have a row for it, print the missing page numbers
# First page is "1" and last page is "8130"
# (this ignores secret pages and only addresses digit strings, that's okay)
for i in range(1, 8131):
    if str(i) not in page_summaries_df['page_id'].values:
        print(f"Missing page: {i}")

# If everything works fine we should only be missing [2399, 3038, 3088, 7902, 7903, 7904], the Six Official Missing Pages

Missing page: 2399
Missing page: 3038
Missing page: 3088
Missing page: 7902
Missing page: 7903
Missing page: 7904


In [67]:
# We can merge the page summaries alone into mspa_df_with_pov, since putting the act/part transcripts on every single row would be a bit much
# We'll merge on vizPageNumber and page_id
# We need to convert mspa_df_with_pov['vizPageNumber'] to a string to merge
mspa_df_with_pov['vizPageNumber'] = mspa_df_with_pov['vizPageNumber'].astype(str)

# Rename the page_id column to vizPageNumber
page_summaries_df.rename(columns={'page_id': 'vizPageNumber'}, inplace=True)

# Merge the data into mspa_df_with_pov
mspa_df_with_pov_and_pages = mspa_df_with_pov.merge(page_summaries_df, on='vizPageNumber', how='left')

# Show some data with page summaries
mspa_df_with_pov_and_pages[mspa_df_with_pov_and_pages['page_group_summary'].notnull()].sample(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands,page_group_summary,act,part
6141,story,005069,"Jade: Combine mecha, streetsweeper, and iron s...",You still have this MECHA GHOST IMAGE you made...,1293261565,/storyfiles/hs2/03166_2.gif,[005070],005068,,[],...,NaN,NaN,NaN,NaN,[],[Jade],"[[Jade: Prepare for Christmas, Jade: Be given ...",Jade alchemizes Duttlemas.,a5a2,part-2
2686,story,002186,IGNORE THIS WOMAN'S ANTICS.,"You're not sure you even saw a woman, let alon...",1245652026,/storyfiles/hs2/00286.gif,[002187],002185,,[],...,NaN,NaN,NaN,Alert : ▪ ▪ ▪,"[Act 2, Part 1, John's House, House, John Egbe...","[John, Rose, Warweary Villein]","[[John: Second tier prototype sprite], [Rose: ...",Dave pesters John about Armageddon.\nRose drop...,act-2,part-1
4832,story,003903,Karkat: Leave your room.,"You step outside your respiteblock, onto one o...",1276482930,/storyfiles/hs2/02003_1.gif,[003904],003902,,[],...,NaN,NaN,NaN,NaN,[],[Karkat],"[[Karkat: Be introduced, Karkat: Talk to Gamzee]]",Karkat Vantas examines his block.,a5a1,part-2
7561,story,006310,==>,,1325388647,/storyfiles/hs2/04407.gif,[006311],006309,,[],...,NaN,NaN,NaN,NaN,[],[Jane],"[[Jane: Do some sorta lifey thing, Jane: Wake ...",Jack Noir assassinates Dream Jane and escapes ...,a6-2,part-3
2284,story,001854,EPILOGUE -> Next.,,1237863890,/advimgs/ps/ps1640.gif,[001855],001853,retro,[],...,NaN,NaN,NaN,NaN,[],[],[],John makes an unadulterated rocket pack.,act-4,part-1
2246,story,001833,Next.,,1236728643,/advimgs/ps/ps1619.gif,[001834],001832,retro,[],...,NaN,NaN,NaN,NaN,[],[],[],"The Mendicant remembers the White Queen, who g...",act-4,part-1
10927,story,009469,[A6A6I5] ====>,,1432594410,/storyfiles/hs2/07565.gif,[009470],009468,,[],...,NaN,NaN,NaN,NaN,[],"[Calliope, Jade, Jane]","[[Calliope: Fall through spiral, Calliope: Arr...",The Skaian cloud vision of the kids at the lil...,a6a6-5,part-4
7398,story,006152,==>,,1322725714,/storyfiles/hs2/04249.gif,[006153],006151,,[],...,NaN,NaN,NaN,NaN,[],[Jake],"[[Jake: Talk with Jane, Jake: Be told about de...",An Old Goat surfaces.,a6-1,part-3
10739,story,009282,[A6A6I4] ====>,"|DIALOGLOG|<br /><span style=""color: #0715cd"">...",1421211413,/storyfiles/hs2/07379.gif,[009283],009281,,[],...,NaN,NaN,NaN,pat pat,[],"[Gamzee, John, Kanaya, Karkat, Sollux, Terezi,...","[[Gamzee: FL1P], [John: FL1P, John: Knock Vris...","Karkat, Kanaya, and Sollux arrive on the roof....",a6a6-4,part-4
239,story,000226,Call the royal locksmith!,The most plainly obvious course of action is t...,1205201884,/advimgs/ps/ps008.gif,[000227],000225,retro,[],...,NaN,NaN,NaN,PS: ?,[],[],[],Act 1.\nWho is this young man?\nJohn Egbert ex...,act-1,part-1


# Adding MSPA commentary data

From the books, we have commentary for groups of pages in Problem Sleuth and Homestuck that might be a bit pointless for prediction models, but is still useful text data.

In [68]:
"""
MSPA_COMMENTARY is a json export of an SQL table that contains the per-Viz-page commentary (by the creator Andrew Hussie) and notes (by me and other volunteers, like fanmade commentary).
There are two tables, one for Homestuck ("commentary") and another for Problem Sleuth ("problem-sleuth") with this format:
```json
[
{
    "type": "table",
    "name": "story",
    "database": "commentary",
    "data": 
[
    {
        "id":"1",
        "title":"Homestuck",
        "page":"7",
        "commentary":"Homestuck (the stable release) began on April 13th (i.e. 4\/13), 2009. My intent was always to make the start date a very significant number in the story, recurring frequently. Consequently, I decided to make him thirteen years old, thus making the story about four thirteen-year-old kids. There are more references to this number than can be mentioned casually, some of which are serendipitous. Playing cards, which have a good deal of relevance later, are comprised of four suits of thirteen cards each, for instance.",
        "notes":null
     },
...
{
    "type": "table",
    "name": "problem-sleuth",
    "database": "commentary",
    "data": 
[
{
    "id": "1",
    "title": "Problem Sleuth",
    "page": "15",
    "commentary": "The title \"Problem Sleuth\" resulted from a quest to determine the most awkward and peculiar term for a detective imaginable. I ran a bunch of other possibilities by my friend, and she instantly green-lighted PS when I said it. Little did I know I would find a way to top that phrase when introducing the tall skinny one a few hundred pages later. Though his name probably wouldn't have made as good a title, admittedly.",
    "notes": null
},
...
```
It should be fairly trivial to turn this into a dataset that we can merge based on the adventure title ("vizStory") and page number ("id"->"vizPageNumber", the "page" is the physical book page). Then, we can keep the page_commentary and page_notes columns.
"""

# Load the data
with open(MSPA_COMMENTARY, 'r', encoding="utf-8") as f:
    commentary_data = json.load(f)

hs_data = commentary_data[0]['data']
ps_data = commentary_data[1]['data']

# Create DataFrames
hs_df = pd.DataFrame(hs_data)
ps_df = pd.DataFrame(ps_data)

# Fill the vizStory column with each title on each dataframe
hs_df['vizStory'] = 'Homestuck'
ps_df['vizStory'] = 'Problem Sleuth'

# Concatenate the DataFrames
commentary_df = pd.concat([hs_df, ps_df], ignore_index=True)

# We don't care about the title or the page (book page), we care about id (renamed to vizPageNumber), the commentary and notes
commentary_df = commentary_df[['vizStory', 'id', 'commentary', 'notes']]
commentary_df.rename(columns={'id': 'vizPageNumber', 'commentary': 'pageCommentary', 'notes': 'pageNotes'},
                     inplace=True)

# Show some data
commentary_df.sample(10)

,vizStory,vizPageNumber,pageCommentary,pageNotes
2938,Homestuck,2940,The final station plops right on top of the be...,None
4894,Problem Sleuth,1182,None,None
892,Homestuck,894,None,None
692,Homestuck,693,None,None
3229,Homestuck,3231,None,None
3210,Homestuck,3212,It's hard to pinpoint exactly why Jade spontan...,None
761,Homestuck,763,None,None
5215,Problem Sleuth,1503,I wonder how Hawking would react to this page....,None
1924,Homestuck,1926,I think the important takeaway here is that DD...,None
3046,Homestuck,3048,None,None


In [69]:
# Merge the data into mspa_df_with_pov
mspa_df_with_pov_and_commentary = mspa_df_with_pov_and_pages.merge(commentary_df, on=['vizStory', 'vizPageNumber'],
                                                                   how='left')

# Show some data with notes
mspa_df_with_pov_and_commentary[mspa_df_with_pov_and_commentary['pageNotes'].notnull()].sample(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,transcript_tag,transcript,tags,pov_characters,pov_commands,page_group_summary,act,part,pageCommentary,pageNotes
2092,story,001722,Next.,Due to principles of uncertainty regarding the...,1235292920,/advimgs/ps/ps1509.gif,[001723],001721,retro,[],...,NaN,NaN,[],[],[],"Dave alchemizes his pre-punched item, a cruxit...",act-4,part-1,I know of at least one person in the universe ...,Makin: And here it is: https://www.instagram.c...
3328,story,002733,[S] MIDNIGHT CREW: ACT 1031,,1257152645,/storyfiles/hs2/00833/00833.swf,[002734],002732,,"[F, HQ, BOLIN, BOLINHQ]",...,NaN,NaN,[],"[Biscuits, Cans, Clover, Clubs Deuce, Crowbar,...","[[Biscuits: Midnight Crew: Act 1031], [Cans: M...",Cinematic 19: Midnight Crew.\nJade pesters Dav...,act-3,part-1,"The Midnight Crew have item/weapon duality, ju...",Makin: 1031 is just a reference to the flash b...
4435,story,003620,[S] Rose and Dave: Shut up and jam.,Dave shows you some of his sweet gear. Wow he ...,1271898902,/storyfiles/hs2/01720/01720.swf,[003621],003619,,"[F, BOLIN]",...,NaN,NaN,[],"[Dave, Lil' Cal, Rose]","[[Dave: Jam with Rose, Dave: See Rose throw dr...",Kanaya bugs Sollux to turn on her viewport.\nP...,act-4,part-1,This is a montage of Dave and Rose dancing to ...,"Makin: This flash originally used the songs ""B..."
6748,story,005614,Exit.,,1302748513,/storyfiles/hs2/03711/03711.swf,[005615],005613,,[F],...,NaN,NaN,[],[Terezi],"[[Terezi: Find Nepeta's body, Terezi: Eject di...",Terezi dwells on the necessity of taking Vrisk...,a5a2,part-2,That's it for exactly two years of Homestuck. ...,Makin: That's it for the commentary mod of the...
2368,story,001924,John: Examine incoming message.,You pull up to your COMPUTER. This is where yo...,1239779693,/storyfiles/hs2/00024_1.gif,[001925],001923,,[],...,NaN,NaN,"[Part 1, Act 1, John's House, John's Room, Com...",[John],"[[John: Be introduced, John: Mess around]]",Dave pesters John about loot and apple juice.,act-1,part-1,None,Makin: The ~ATH file on John's desktop actuall...
3325,story,002730,[S] Jade: Open FreshJamz!,You open the FRESHJAMZ MEDIA PLAYER and add Da...,1256977886,/storyfiles/hs2/00830/00830.swf,[002731],002729,,"[F, BOLIN]",...,NaN,NaN,"[Part 1, Act 3]",[Jade],"[[Jade: Be introduced, Jade: Talk to Dave]]",Pseudogame 9: FreshJamz.,act-3,part-1,"With the (playable!) FRESHJAMZ playlist, we ar...","Makin: This flash originally used the songs ""R..."
3702,story,003023,==>,"<a href=""http://www.mspaintadventures.com/swee...",1262649310,/storyfiles/hs2/01123_2.gif,[003024],003022,,[],...,NaN,KLONK,"[Part 1, Act 3, Lined Mode, Carapacian, Dersit...","[Authority Regulator, Parcel Mistress, Warwear...",[[Authority Regulator: Open fire with missile ...,The Renegade shoots rockets all over the place...,act-3,part-1,"Subtle trivia: Did you know the ""LoB"" sound ef...",Drew: I don't know about pressing it with your...
2743,story,002238,[S] ==>,Left knob: volume for current sample. <br />Ri...,1246679697,/storyfiles/hs2/00338/00338.swf,[002239],002237,,"[F, BOLIN]",...,NaN,NaN,"[Act 2, Part 1]",[Dave],"[[Dave: Play sick beats, Dave: Have game stole...",Dave browses the web. Rose pesters him about h...,act-2,part-1,"On the beat sequencer page, you could make you...",Drew: The Ghostbusters theme was also among th...
4064,story,003314,==>,You decide to give her the chalk. She is grate...,1266530135,/storyfiles/hs2/01414.gif,[003315],003313,,[],...,NaN,-.-- .- -.-- !!!,"[Part 1, Act 4, Carapacian, Exile, PM, Dersite...","[Parcel Mistress, Warweary Villein]","[[Parcel Mistress: Talk to John], [Warweary Vi...",The Renegade retrieves the refrigerator and co...,act-4,part-1,None,Serenity: Yay!!!
3551,story,002922,Rose: Build as much as you can as fast as you ...,"|PESTERLOG|<br /><span style=""color: #0715cd"">...",1261008164,/storyfiles/hs2/01022.gif,[002923],002921,,[],...,NaN,NaN,"[Part 1, Act 3, John Egbert, John's House, Alc...","[John, Rose]","[[John: Start building 

# Cleaning up

We plan on converting this to CSV and potentially JSONL, so right now have a lot of object columns that should be strings, a bunch of empty strings and NaNs that should be pd.NAs. 

In [70]:
# mspa_df_with_pov currently has these columns representing MSPA data:
# section (str, per page): the section of the MSPA website the page is in, always a string like "story"
# pageId (str, per page): the page number, usually a six digit padded string like "001901", sometimes a regular string like "pony"
# title (str, per page): the title of the page, usually the command given in the previous page, always a string like "John: Enter name."
# content (str, per page): the content of the page, usually the page minus the panels (includes dialog and HTML)
# timestamp (timestamp, per page): the UNIX timestamp of the page, always a number like 1580000000
# src (str, per panel): the partial image file name, when prefixed with mspaintadventures.com or homestuck.com, something like "/storyfiles/hs2/001901.gif"
# next (list of strings, per page): the pageId of the next pages, usually a list of one string like ["001902"]
# previous (string, per page): the pageId of the previous page, usually a string like "001900"
# theme (str, per page): the theme of the page, like "A6A6", always a string, usually empty
# flag (list of strings, per page): a list of flags for Homestuck Collection presentation, some flags like ["R1"] show that the page is part of a "retcon" where Hussie went back and edited panels, others like ["FIREFLY"] show the page has special javascript that shows fireflies flying around the page, a page usually has no flag
# vizStory (str, per page): essentially a readable version of the section, always a string like "Homestuck"
# vizPageNumber (str, per page): pages are successive in MSPA but per section in VIZ, for example Homestuck's first page is 1901 in MSPA but 1 in VIZ, always a non-padded string since string pages like "pony" still exist
# vizPageId (str, per page): we should remove this column, being a combination of section number and viz page that was only used in one dataset we wanted to merge with
# vizLink (str, per page): the link to the page in the VIZ website, always a full URI like "https://homestuck.com/story/5320"
# pageNumber (str, per page): like vizPageId but for MSPA format, also should be removed
# transcript class (str, per panel): the class of the transcript, like "dialogue" or "narration", always a string, usually a NaN which should be replaced with pd.NA
# transcript style (str, per panel): the style of the transcript, in CSS, like "font-weight: bold", always a string, usually a NaN which should be replaced with pd.NA
# transcript tag (str, per panel): the tag of the transcript, like "p" or "div", always a string, usually a NaN which should be replaced with pd.NA
# transcript (str, per panel): the transcript of the panel, the text that appears in the panel, always a string, usually a NaN which should be replaced with pd.NA
# tags (list of strings, per page): the tags of the page, like ["John Egbert", "Rose Lalonde", "Land of Wind and Shade"], always a list of strings, usually an empty list
# pov_characters (list of strings, per page): the characters that appear in the page according to the POV cam, always a list of strings, usually an empty list
# pov_commands (list of strings, per page): the commands that the characters have in the page according to the POV cam, always a list of lists of strings (per character), usually an empty list
# pageCommentary (string, per page): the commentary of the page, usually by Andrew Hussie, always a string, usually a NaN or None which should be replaced with pd.NA
# pageNotes (string, per page): the notes of the page, usually by volunteers, always a string, usually a NaN or None which should be replaced with pd.NA
# page_group_summary (string, per group of pages): the summary of an arbitrary but small group of pages, or a scene, as decided and described by Rafe from rafe.name, always a string, should be replaced with pd.NA if empty
# act (str, per page): the act the page is in, always a string like "act-1"
# part (str, per page): the part the page is in, always a string like "part-1"

# Let's start by removing the columns we don't need
mspa_df_cleaned = mspa_df_with_pov_and_commentary.drop(columns=['vizPageId', 'pageNumber'])

# Replace NaN with pd.NA in the transcript columns
na_to_pdna_columns = ['transcript_class', 'transcript_style', 'transcript_tag', 'transcript', 'pageCommentary',
                      'pageNotes', 'page_group_summary', 'act', 'part']
mspa_df_cleaned[na_to_pdna_columns] = mspa_df_cleaned[na_to_pdna_columns].where(
    mspa_df_cleaned[na_to_pdna_columns].notnull(), pd.NA)

# Replace empty strings with pd.NA in all columns
mspa_df_cleaned = mspa_df_cleaned.replace('', pd.NA)

# Show some cleaned data
mspa_df_cleaned.sample(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,transcript_tag,transcript,tags,pov_characters,pov_commands,page_group_summary,act,part,pageCommentary,pageNotes
1577,story,001333,HD: To the Fromagere District!!,You back through the SQUARE SHOULDER TOLLS in ...,1226374252,/advimgs/ps/ps1120.gif,[001334],001332,retro,[],...,<NA>,<NA>,[],[],[],The Renegade plays courthouse.\nJade places Jo...,act-3,part-1,<NA>,<NA>
5419,story,004403,======>,This moon is different. It's very... <br /><br...,1282528739,/storyfiles/hs2/02502.gif,[004404],004402,<NA>,[],...,<NA>,<NA>,[],[Sollux],[[Sollux: Wake on moon of Derse]],The mounted Gmork head scares Hussie shitless....,a5a1,part-2,<NA>,<NA>
9621,story,008198,[A6A6I1] ====>,<NA>,1371523406,/storyfiles/hs2/06296.gif,[008199],008197,<NA>,[],...,<NA>,<NA>,[],"[Jade, Roxy]","[[Jade: Intimidate Roxy, Jade: Tell Roxy about...",Jade tells Roxy to make the Matriorb.,a6a6-1,part-4,<NA>,<NA>
3935,story,003223,[I] ==>,"<span style=""color: #2ed73a"">Clover</span> ins...",1265491703,/storyfiles/hs2/01323.gif,[003224],003222,<NA>,[],...,<NA>,CLOVER: !,"[Part 1, Intermission, The Felt, Clover, Felt ...","[Clover, Diamonds Droog, Hearts Boxcars, Spade...","[[Clover: Warn SS of dangers of opening safe, ...","Clover riddles the scofflaws about the safe, s...",intermission,part-1,<NA>,<NA>
7530,story,006283,[A6I1] ==>,"|DIALOGLOG|<br /><span style=""color: #e00707"">...",1324711795,/storyfiles/hs2/04380.gif,[006284],006282,<NA>,[],...,<NA>,<NA>,[],"[Aradia, Dave, Kanaya, Karkat, Parcel Mistress...","[[Aradia: Mayor arrives, Aradia: Give meteor p...",The Monarch sends Serenity and the Vagabond to...,a6-1,part-3,<NA>,<NA>
4881,story,003941,======>,"The prosecution begs your pardon, dear senator...",1276821069,/storyfiles/hs2/02041_2.gif,[003942],003940,<NA>,[],...,<NA>,<NA>,[],[Terezi],"[[Terezi: Introduction, Terezi: Cr1m1n4l, cr1m...",Terezi Pyrope plays Courtblock Drama.,a5a1,part-2,"At the same time, it's just as likely for legi...",<NA>
3154,story,002587,==>,Mayors are so much better than kings. You hate...,1253765070,/storyfiles/hs2/00687.gif,[002588],002586,<NA>,[],...,<NA>,<NA>,"[Part 1, Act 2, Hero Mode, WV, BK, Carapacian,...",[Warweary Villein],"[[Warweary Villein: WV: Retri..., Ascend.]]",The Vagabond presides over Can Town and eats v...,act-2,part-1,Why does WV hate kings?? He has a troubled and...,<NA>
6617,story,005504,==>,"|PESTERLOG|<br /><span style=""color: #a10000"">...",1298979441,/storyfiles/hs2/03601_2.gif,[005505],005503,<NA>,[],...,<NA>,S4D3HH3 H34DS A433H33H44...,[],"[Aradia, Jade, Kanaya, Tavros, Terezi]","[[Aradia: ======>], [Jade: Meet Aradia, Aradia...",Kanaya joins the bubble and relives restoring ...,a5a2,part-2,<NA>,<NA>
5489,story,004470,======>,<NA>,1283384242,/storyfiles/hs2/02569.gif,[004471],004469,<NA>,[],...,<NA>,PSSWOOOP!,[],[Terezi],"[[Terezi: Psswoop, Terezi: Talk with Dragonspr...",Terezi psswooops with Dragonsprite.,a5a1,part-2,<NA>,<NA>
4769,story,003861,==>,"The toy has taken a new master. The tactician,...",1276065696,/storyfiles/hs2/01961_1.gif,[003862],003860,<NA>,[],...,<NA>,<NA>,"[Part 1, Act 4, Jack Noir, Carapacian, Dersite]","[Jack Noir, John]","[[Jack Noir: Exchange with PM, gift to John], ...","Dream John opens the package, finding letters ...",act-4,part-1,"At last, the true contents of the box, reveale...",<NA>


In [71]:
# Fix the inconsistent casing (some columns are snake_case, others camelCase)
# Annoyingly, there doesn't seem to be a standard for this for ML datasets, so we'll go with Python's snake_case
# This means altering pageId, vizStory, vizPageNumber, vizLink, pageCommentary, pageNotes

mspa_df_cleaned.rename(columns={
    'pageId': 'page_id',
    'vizStory': 'viz_story',
    'vizPageNumber': 'viz_page_number',
    'vizLink': 'viz_link',
    'pageCommentary': 'page_commentary',
    'pageNotes': 'page_notes'
}, inplace=True)

mspa_df_cleaned.sample(5)

,section,page_id,title,content,timestamp,src,next,previous,theme,flag,...,transcript_tag,transcript,tags,pov_characters,pov_commands,page_group_summary,act,part,page_commentary,page_notes
9483,story,008070,[A6I5] ==>,<NA>,1365569059,/storyfiles/hs2/06167.gif,[008071],008069,<NA>,[],...,<NA>,<NA>,[],[John],"[[John: Jump around space, time]]",John continues zapping around. He sees Vriska ...,a6-5,part-3,<NA>,<NA>
1064,story,000885,Thugs: AGGRESS,The thugs take you to task with an array of BU...,1219549728,/advimgs/ps/ps673.gif,[000886],000884,retro,[],...,<NA>,\nPLUCK RELIQUARY\n\nBONK\n,[],[],[],Cinematic 14: Dave Ascends.\nThe Vagabond make...,act-2,part-1,<NA>,<NA>
9808,story,008387,==>,<NA>,1377798543,/storyfiles/hs2/06484.gif,[008388],008386,A6A6,[],...,<NA>,<NA>,[],"[Caliborn, John]","[[Caliborn: Continue Homosuck, Caliborn: Be in...",The real John intrudes on Homosuck. Caliborn t...,a6a6-2,part-4,<NA>,<NA>
5385,story,004362,======>,"And while yet another partook not in revelry, ...",1282162250,/storyfiles/hs2/02461.gif,[004363],004361,<NA>,[],...,<NA>,<NA>,[],"[Eridan, Feferi]","[[Eridan: Talk to Vriska], [Feferi: Do somethi...",Feferi quiets Gl’bgolyb’s terrible voice with ...,a5a1,part-2,<NA>,<NA>
831,story,000684,Next.,Suddenly you flip out!!! Wielding the TEDDY BE...,1214181597,/advimgs/ps/ps467.gif,[000685],000683,retro,[],...,<NA>,<NA>,[],[],[],Rose deploys the Punch Designix for John.,act-2,part-1,There is only one adverb that can possibly app...,<NA>


In [72]:
# Make column types explicit as verification and so they don't resolve to float when we save them, and timestamp stays a timestamp

mspa_df_cleaned['timestamp'] = pd.to_datetime(mspa_df_cleaned['timestamp'], unit='s')

# Convert all columns to string except for the lists and timestamp

for column in mspa_df_cleaned.columns:
    if column not in ['timestamp', 'next', 'tags', 'pov_characters', 'pov_commands', 'flag']:
        mspa_df_cleaned[column] = mspa_df_cleaned[column].astype('string')

mspa_df_cleaned.sample(5)

C:\Users\PC\AppData\Local\Temp\ipykernel_14788\4222095517.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  mspa_df_cleaned['timestamp'] = pd.to_datetime(mspa_df_cleaned['timestamp'], unit='s')


,section,page_id,title,content,timestamp,src,next,previous,theme,flag,...,transcript_tag,transcript,tags,pov_characters,pov_commands,page_group_summary,act,part,page_commentary,page_notes
10870,story,009404,[A6A6I5] ====>,"|DIALOGLOG|<br /><span style=""color: #b536da"">...",2015-05-06 23:30:24,/storyfiles/hs2/07500.gif,[009405],009403,<NA>,[],...,<NA>,<NA>,[],"[Dave, Jaspersprite, Rose, Rosesprite, Roxy, V...","[[Dave: Talk with Rose, Roxy, Dave: Meet Roses...",Dave and Roxy grill each other’s asses. Jasper...,a6a6-5,part-4,<NA>,<NA>
309,story,000065,"With great prescience, move away from end of t...","With great prescience, you move away from the ...",2007-06-04 05:41:19,/advimgs/jb/mspaintadventure55c.gif,[000066],000064,retro,[],...,<NA>,SMASH,[],[],[],John attaches fake arms to his gift harlequin....,act-1,part-1,<NA>,<NA>
2475,story,002010,[S] John: Check Pesterchum.,|PESTERLOG|<br />-- gardenGnostic <span style=...,2009-05-03 08:19:00,/storyfiles/hs2/00110/00110.swf,[002011],002009,<NA>,[F],...,<NA>,<NA>,"[Part 1, Act 1]","[Dave, John]",[[Dave: Tell John to bone up on data structure...,Dave pesters John about terrible movie stars.,act-1,part-1,McConaughey does sound like a noise a horse wo...,<NA>
1148,story,000954,Death: COMBAT OPERANDI -> TWO LUMPS,Sugar is offered with aggressive courtesy! Dea...,2008-09-03 06:09:52,/advimgs/ps/ps742_2.gif,[000955],000953,retro,[],...,<NA>,<NA>,[],[],[],The Server Room is also locked. The Vagabond e...,act-2,part-1,It just occurred to me the fact that Death doe...,<NA>
3414,story,002808,John: Resist great urge to take the wedged shale.,"You know you should grab this thing, but...",2009-11-24 21:12:44,/storyfiles/hs2/00908.gif,[002809],002807,<NA>,[],...,<NA>,<NA>,"[Part 1, Act 3, John Egbert, Grist, Roof]",[John],"[[John: <==, John: Enter Dad's room]]",John and Jade steel themselves for familial en...,act-3,part-1,John: Never mind that big purple Gusher.,<NA>


In [73]:
# Let's reorder the columns to make it easier to read, the first columns should let a user know the story and the title of the page at least

mspa_df_cleaned = mspa_df_cleaned[
    ['viz_story', 'title', 'content', 'page_id', 'timestamp', 'src', 'next', 'previous', 'act', 'part', 'viz_page_number', 'viz_link',
     'page_group_summary', 'tags', 'pov_characters', 'pov_commands', 'transcript_class', 'transcript_style',
     'transcript_tag', 'transcript', 'page_commentary', 'page_notes', 'theme', 'flag']]

mspa_df_cleaned.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,act,part,...,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
2691,Homestuck,Rose: Pick up car.,<NA>,002190,2009-06-22 22:37:35,/storyfiles/hs2/00290.gif,[002191],002189,act-2,part-1,...,"[John, Rose, Warweary Villein]","[[John: Second tier prototype sprite], [Rose: ...",<NA>,<NA>,<NA>,CONNECTION LOST,<NA>,<NA>,<NA>,[]
3274,Homestuck,==>,And today will likely be no exception. Among t...,002691,2009-10-21 03:43:56,/storyfiles/hs2/00791.gif,[002692],002690,act-3,part-1,...,[Jade],"[[Jade: Be introduced, Jade: Talk to Dave]]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
7852,Homestuck,[A6I2] ==>,"|PESTERLOG|<br /><span style=""color: #626262"">...",006585,2012-03-12 00:19:02,/storyfiles/hs2/04682_2.gif,[006586],006584,a6-2,part-3,...,"[Gamzee, Karkat, Rose, Terezi]","[[Gamzee: Year 1], [Karkat: Talk to self], [Ro...",<NA>,<NA>,<NA>,* | 1 | 2 | 3 | * | | | | *...,<NA>,<NA>,<NA>,[]
10009,Homestuck,[A6A6I2] ====>,"|DIALOGLOG|<br /><span style=""color: #929292"">...",008574,2013-09-17 04:11:53,/storyfiles/hs2/06671.gif,[008575],008573,a6a6-2,part-4,...,"[Calliope, Jade]","[[Calliope: Be visited by Jade], [Jade: ==>, J...",<NA>,<NA>,<NA>,CALLIOPE: ?,<NA>,<NA>,<NA>,[]
152,Jailbreak,Grab the other guy's hand and run off into the...,Yes! Finally! Free at last!<br /><br />Though ...,000110,2007-06-04 20:19:06,/advimgs/jb/lv3/mspaintadventure23h.gif,[000111],000109,act-1,part-1,...,[],[],<NA>,<NA>,<NA>,CH-CLICK.,<NA>,<NA>,retro,[]


In [74]:
# And let's make a dict with column descriptions for later when we release the dataset

dataset_column_descriptions = {
    'viz_story': "The story the page is part of, always 'Homestuck'.",
    'title': "The title of the page, usually the command given in the previous page.",
    'content': "The content of the page, usually the page minus the panels (includes dialog and HTML).",
    'page_id': "The page number, usually a six digit padded string like '001901', sometimes a regular string like 'pony'.",
    'timestamp': "The UNIX timestamp of the page.",
    'src': "The partial image file name, when prefixed with mspaintadventures.com or homestuck.com, something like '/storyfiles/hs2/001901.gif'.",
    'next': "The pageId of the next pages, usually a list of one string like ['001902'].",
    'previous': "The pageId of the previous page, usually a string like '001900'.",
    'act': "The act the page is in, always a string like 'act-1'.",
    'part': "The part the page is in, always a string like 'part-1'.",
    'viz_page_number': "Pages are successive in MSPA but per section in VIZ, for example Homestuck's first page is 1901 in MSPA but 1 in VIZ.",
    'viz_link': "The link to the page in the VIZ website.",
    'page_group_summary': "The summary of the group of pages this page is in.",
    'tags': "The tags of the page, like ['John Egbert', 'Rose Lalonde', 'Land of Wind and Shade'].",
    'pov_characters': "The characters that appear in the page according to the POV cam.",
    'pov_commands': "The commands that the characters have in the page according to the POV cam.",
    'transcript_class': "The class of the transcript, like 'dialogue' or 'narration'.",
    'transcript_style': "The style of the transcript, in CSS.",
    'transcript_tag': "The tag of the transcript, like 'p' or 'div'.",
    'transcript': "The transcript of the panel, the text that appears in the panel.",
    'page_commentary': "The commentary of the page, usually by Andrew Hussie.",
    'page_notes': "The notes of the page, usually by volunteers.",
    'theme': "The theme of the page, like 'A6A6'.",
    'flag': "A list of flags for Homestuck Collection presentation."
}

In [75]:
# Before we continue we need to confirm we can fetch the image or asset of each panel
# Pick an asset with a src, like "/storyfiles/hs2/001901.gif"

# We'll use a random homestuck panel with enough metadata, like the one in viz page 2519
panel_src = mspa_df_cleaned[mspa_df_cleaned['viz_page_number'] == '2519']['src'].values[0]
panel_url = f"https://www.homestuck.com/images{panel_src}"

# Test the image URL
print(f"Fetching image from {panel_url}")
requests.get(panel_url)


Fetching image from https://www.homestuck.com/images/storyfiles/hs2/02518.gif


<Response [200]>

In [76]:
Image(url=panel_url)

In [85]:
# Right now we haven't even saved the data, so let's do that first, as csv

mspa_dataset_output_file = os.path.join(OUTPUT_FOLDER, 'mspa_dataset.csv')
mspa_df_cleaned.to_csv(mspa_dataset_output_file, index=False)

# Now we can load the data back (CSV does have the limitation of not representing pd.NA correctly though, we'll just get NaNs)
mspa_df_csv = pd.read_csv(mspa_dataset_output_file)
mspa_df_csv.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,act,part,...,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
4597,Homestuck,Dave: Combine camera and captchalogue card.,You make the CAPTCHAROID CAMERA.<br /><br />Yo...,003741,2010-05-14 04:52:16,/storyfiles/hs2/01841_1.gif,['003742'],3740.0,act-4,part-1,...,['Dave'],[['Dave: Wake up and jam.']],NaN,NaN,NaN,"\nCAPTCHAROID CAMERA\n⬡ 5,000 ○ 500 □ 500 ...",Another actual useful item is created. The Cap...,NaN,NaN,[]
9303,Homestuck,[A6I5] ==>,"|DIALOGLOG|<br /><span style=""color: #005682"">...",007893,2013-03-14 10:48:18,/storyfiles/hs2/05990.gif,['007894'],7892.0,a6-5,part-3,...,"['Aradia', 'Aranea', 'Caliborn', 'Feferi', 'Jo...","[[""Aradia: Be in Vriska's crew""], [""Aranea: Jo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
5781,Homestuck,==>,NaN,004745,2010-11-08 00:44:34,/storyfiles/hs2/02844.gif,['004746'],4744.0,a5a2,part-2,...,['Jack Noir'],"[[""Jack Noir: Take Bro's sword from Beat Mesa""]]",NaN,NaN,NaN,NaN,We last saw Bro and Jack dueling on this recor...,NaN,NaN,[]
2520,Homestuck,TT: Select bunny.,NaN,002046,2009-05-13 02:20:28,/storyfiles/hs2/00146.gif,['002047'],2045.0,act-1,part-1,...,"['John', 'Rose']","[['John: Get trapped in room'], ['Rose: Connec...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
9838,Homestuck,==>,NaN,008408,2013-08-31 20:52:52,/storyfiles/hs2/06505.gif,['008409'],8407.0,a6a6-2,part-4,...,"['Caliborn', 'John']","[['Caliborn: Continue Homosuck', 'Caliborn: Be...",NaN,NaN,NaN,HORSE ACCIDENT.,NaN,NaN,A6A6,[]


# Tagging panels

We have the panel images, we have the ReadMSPA transcripts of them, we have the point of view from the POV cam, and we have the title and text that accompanies the panels. We have some partial tagging information from the Homestuck Search engine. With that and a vision model, we might be able to successfully extract non-hallucinated information.
Basically we need to predict the tag-related columns, since we only have tagging data for the first four acts.

In [78]:
# First we need to get only the image we need. We can assemble the image URLs from the src column, by prefixing "https://homestuck.com/images" to it
# That, plus the rest of informational columns, will be useful information to train the model with
# We want to predict the "tags" column given the rest, so we take the Act 1-4 data (basically everything that has a non-empty tags column):

four_acts_hs_df = mspa_df_cleaned[mspa_df_cleaned['tags'].apply(len) > 0]
four_acts_hs_df.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,act,part,...,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
4252,Homestuck,==>,Looks like it worked! You love your scribblepad.,003466,2010-03-07 21:49:55,/storyfiles/hs2/01566.gif,[003467],003465,act-4,part-1,...,"[Dave, Jade]","[[Dave: Perform alchemy, Dave: Upgrade alchemi...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
4247,Homestuck,Dave: Use a punched Gamebro Magazine card.,The ALCHEMITER is upgraded with a huge metal b...,003460,2010-03-07 02:31:21,/storyfiles/hs2/01560_2.gif,[003461],003459,act-4,part-1,...,"[Dave, Jade]","[[Dave: Perform alchemy, Dave: Upgrade alchemi...",<NA>,<NA>,<NA>,<NA>,"Okay, that's it. ""Dave: Use a punched GameBro ...",<NA>,<NA>,[]
4166,Homestuck,"John, the uncarved tablet you retrieved.","Why yes, it appears you do.",003389,2010-02-25 21:23:46,/storyfiles/hs2/01489.gif,[003390],003388,act-4,part-1,...,"[John, Parcel Mistress]","[[John: Put out fire, John: Be commanded by PM...",<NA>,<NA>,<NA>,"> John, the uncarved tablet you retrieved. Do ...",<NA>,<NA>,<NA>,[]
3529,Homestuck,WV: Appearify the temple.,That's such a dumb idea. Not as dumb as using ...,002908,2009-12-13 01:17:18,/storyfiles/hs2/01008.gif,[002909],002907,act-3,part-1,...,[Warweary Villein],[[Warweary Villein: WV: Descend.]],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
3044,Homestuck,==>,Where'd the little dude scamper off to this time?,002499,2009-09-06 04:24:02,/storyfiles/hs2/00599_2.gif,[002500],002498,act-2,part-1,...,[Dave],[[Dave: Puppets]],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]


In [79]:
# For the dataset we'll need to make a new dataframe with only the relevant data
# We don't need viz_story, timestamp, next, previous, viz_page_number, viz_link
# We'll keep the rest, and remove the tag later when it's time

tagging_dataset = four_acts_hs_df.drop(
    columns=['viz_story', 'timestamp', 'next', 'previous', 'viz_page_number', 'viz_link'])

tagging_dataset.sample(5)

,title,content,page_id,src,act,part,page_group_summary,tags,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
3012,Dave: Transfer katana to strife specibus.,"With an escalating sense of threat, you think ...",002468,/storyfiles/hs2/00568.gif,act-2,part-1,Dave unwittingly stars in Bro’s grisly puppet ...,"[Part 1, Act 2, Dave's House, Kitchen, Syllade...",[Dave],[[Dave: Puppets]],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
3042,==>,Wait...<br /><br />Who's that looking at you i...,002498,/storyfiles/hs2/00598.gif,act-2,part-1,Dave wraps up his kitchen activities.,"[Part 1, Act 2, Lil Cal, Fridge]",[Dave],[[Dave: Puppets]],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
2931,Refrigerator: Level up for slaying the imp.,The REFRIGERATOR skyrockets up the ECHELADDER ...,002403,/storyfiles/hs2/00503.gif,act-2,part-1,"The Vagabond, having gorged on the wisdom of H...","[Part 1, Act 2, Fridge, Drawing]","[John, Rose, Warweary Villein]","[[John: Hammer your way to victory, John: Wond...",<NA>,<NA>,<NA>,<NA>,Sometimes inanimate objects gain levels in HS....,<NA>,<NA>,[]
3455,==>,<NA>,002843,/storyfiles/hs2/00943.gif,act-3,part-1,Jade searches for Bec and the package from John.,"[Part 1, Act 3, Jade Harley, Pacific Island, B...","[Becquerel, Jade]",[[Becquerel: Locate and feed the devilbeast yo...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
4113,==>,<NA>,003349,/storyfiles/hs2/01449_2.gif,act-4,part-1,Dave tricks Jade into waking herself with a Ro...,"[Part 1, Act 4, Jade Harley, Jade's House, Jad...","[Dave, Jade]","[[Dave: Find prototyped kernel], [Jade: Protot...",<NA>,<NA>,<NA>,<NA>,There's a very good chance Karkat saw all this...,<NA>,<NA>,[]


In [104]:
# Art styles are tagged inconsistently in HSSE. Everything that does not have a style tag has the style "Sprite Mode". We'll add that.
# The existing styles we need to search for are "Hero Mode", "Lined Mode" (semisprite), "Homosuck" (homosuck mode), "Hussnasty" (detailed style that is unused in the dataset as of yet), and "Scribble Mode"
# First let's check the tags arrays for the styles, and if it lacks all of them, we'll add "Sprite Mode"

art_styles = ["Sprite Mode", "Hero Mode", "Lined Mode", "Homosuck", "Hussnasty", "Scribble Mode"]

# Use as a df function
def add_sprite_mode_tag_if_other_styles_missing(row: pd.Series) -> pd.Series:
    if not any(style in row['tags'] for style in art_styles):
        row['tags'].append("Sprite Mode")
    return row

tagging_dataset = tagging_dataset.apply(add_sprite_mode_tag_if_other_styles_missing, axis=1)

# Show some data
tagging_dataset.sample(5)

,title,content,page_id,src,act,part,page_group_summary,tags,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag,art_style
4491,John: Enter.,There are many frightening and powerful monste...,003663,/storyfiles/hs2/01763.gif,act-4,part-1,Lil Cal lands on a rocket board. The Regulator...,"[Part 1, Act 4, John Egbert, Suit, Wise Guy Sl...",[John],"[[John: Explore., John: Arrive on meteor, John...",<NA>,<NA>,<NA>,CAULK LICH,<NA>,<NA>,<NA>,[],Sprite Mode
3444,==>,The device generates a fetal PARADOX CLONE of ...,002834,/storyfiles/hs2/00934.gif,act-3,part-1,Rose creates a paradox clone of Jaspers.,"[Part 1, Act 3, Rose Lalonde, Pink Scarf, Squi...",[Rose],"[[Rose: Find appearifier, Rose: Find Jaspers]]",<NA>,<NA>,<NA>,<NA>,"""Whoever was operating this machine in the pas...",<NA>,<NA>,[],Sprite Mode
3140,WV: Take the can labeled BEANS.,"Ok, you take that too.",002574,/storyfiles/hs2/00674.gif,act-2,part-1,The Vagabond cannot open the canned food and d...,"[Part 1, Act 2, WV, Exile, Carapacian, Dersite...",[Warweary Villein],"[[Warweary Villein: WV: Retri..., Ascend.]]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],Sprite Mode
3600,John: Combine ghost suit and Wise Guy book.,You make the WISE GUY SLIME SUIT.<br /><br />T...,002955,/storyfiles/hs2/01055_2.gif,act-3,part-1,John incorporates double-punched cards into hi...,"[Part 1, Act 3, John Egbert, Wise Guy Slime Su...",[John],[[John: Alchemise]],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],Sprite Mode
3078,John: Read book. Be the wise guy.,You never really understood what Caveney's rel...,002529,/storyfiles/hs2/00629_1.gif,act-2,part-1,Reading “Wise Guy” inspires John to carve a to...,"[Part 1, Act 2, Wise Guy, Sprite Mode]",[John],"[[John: Shuffle in captchalogue cards, John: M...",<NA>,<NA>,<NA>,\nHarry Anderson\nWISE GUY\nfrom the street to...,I'm guilty of being absolutely convinced this ...,<NA>,<NA>,[],Sprite Mode


In [105]:
"""
Now let's turn it into OpenAI's JSONL format, an example of which is here:

```jsonl
{
  "messages": [
    { "role": "system", "content": "You are an assistant that identifies uncommon cheeses." },
    { "role": "user", "content": "What is this cheese?" },
    { "role": "user", "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/3/36/Danbo_Cheese.jpg"
          }
        }
      ]
    },
    { "role": "assistant", "content": "Danbo" }
  ]
}
```

Each row of the dataset will turn into something here. 
The system prompt will contain an example of the input data, so let's deal with that later.
The first user content will be the metadata minus the image and the tags.
The second user content will be the image URL, which we can once again get from https://homestuck.com/images plus the src column. No need to base64 encode it.
The assistant content will be the tags column.
"""

system_prompt = """You are a professional image tagger.
Your current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.
You will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more useful information, like character appearances and summaries of page groups.
The image alongside will be the panel itself. Here's an example of the metadata input:
```
{"title": "Jade: Keep going.", "content": "Oh yeah. How could you forget about his stash of DECREPIT MUMMIES.<br /><br />God you hate these things.", "page_id": "002753", "src": "/storyfiles/hs2/00853_1.gif", "page_group_summary": "Rose enters the Hubgrid.\nJade descends through Grandpa\u2019s trophies.", "pov_characters": ["Jade"], "pov_commands": [["Jade: Proceed through house", "Jade: Be trolled by Karkat"]], "transcript_class": null, "transcript_style": null, "transcript_tag": null, "transcript": null, "page_commentary": null, "page_notes": null, "theme": null, "flag": []}
```
Note the pov information applies to the page, not to all the panels in it, and a character can not appear in the panel and simply be part of an online conversation within the content, so it's not a perfect predictor of the character-related tags for the specific panel being tagged, but it can help you discriminate if you don't know the characters.

This is an example of your tagged output, which must be a valid JSON list:
```
["Jade Harley", "Jade's House", "Blue Sun Shirt", "Stairs", "Dead", "Light Symbol", "Sprite Mode"]
```
"""


def create_openai_jsonl_entry(row: pd.Series) -> dict:
    """
    Creates an OpenAI JSONL entry from a row of the tagging dataset.

    Args:
        row (pd.Series): A row of the tagging dataset.

    Returns:
        dict: An OpenAI JSONL entry.
    """
    # We must remove the organizational tags like "Act 1" and "Part 1" from the tags, because
    # the model cannot predict them
    usable_tags = [tag for tag in row['tags'] if "Act " not in tag and "Part " not in tag]
    # Tags will also be stringified so the model can predict them
    usable_tags = json.dumps(usable_tags)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": json.dumps(row.drop('tags').to_dict())},
        {"role": "user", "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": f"https://homestuck.com/images{row['src']}"
                }
            }
        ]},
        {"role": "assistant", "content": usable_tags}
    ]
    return {"messages": messages}


# Test it with one row
create_openai_jsonl_entry(tagging_dataset.iloc[1000])

{'messages': [{'role': 'system',
   'content': 'You are a professional image tagger.\nYour current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.\nYou will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more useful information, like character appearances and summaries of page groups.\nThe image alongside will be the panel itself. Here\'s an example of the metadata input:\n```\n{"title": "Jade: Keep going.", "content": "Oh yeah. How could you forget about his stash of DECREPIT MUMMIES.<br /><br />God you hate these things.", "page_id": "002753", "src": "/storyfiles/hs2/00853_1.gif", "page_group_summary": "Rose enters the Hubgrid.\nJade descends through Grandpa’s trophies.", "pov_characters": ["Jade"], "pov_commands": [["Jade: Proceed through house", "Jade: Be trolled by Karkat"]], "transcript_class": null, "transcri

In [95]:
# OpenAI's platform lets you upload two jsonl files, one with the training data and another with the validation data
# We'll split the dataset into 80% training and 20% validation, but we need to shuffle it first

shuffled_tagging_dataset = tagging_dataset.sample(frac=1, random_state=42)
training_size = int(len(shuffled_tagging_dataset) * 0.8)
training_data = shuffled_tagging_dataset.iloc[:training_size]
validation_data = shuffled_tagging_dataset.iloc[training_size:]

# Remove the training and validation files if they don't already exist
# Remember we're storing them in output, as "hs_tagging_tranining_data.jsonl" and "hs_tagging_validation_data.jsonl"

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# Remove the files if they exist
training_file = os.path.join(OUTPUT_FOLDER, "hs_tagging_training_data.jsonl")
validation_file = os.path.join(OUTPUT_FOLDER, "hs_tagging_validation_data.jsonl")

if os.path.exists(training_file):
    os.remove(training_file)
if os.path.exists(validation_file):
    os.remove(validation_file)

# Create the training and validation jsonl files
with open(training_file, 'w') as training_jsonl, open(validation_file, 'w') as validation_jsonl:
    for _, row in training_data.iterrows():
        training_jsonl.write(json.dumps(create_openai_jsonl_entry(row)) + '\n')
    print(f"Training data written to {training_file}")
    for _, row in validation_data.iterrows():
        validation_jsonl.write(json.dumps(create_openai_jsonl_entry(row)) + '\n')
    print(f"Validation data written to {validation_file}")

# Let's load the first line as a json
with open(training_file, 'r') as f:
    first_line_json = json.loads(f.readline())
    
first_line_json

Training data written to ../output\hs_tagging_training_data.jsonl
Validation data written to ../output\hs_tagging_validation_data.jsonl


{'messages': [{'role': 'system',
   'content': 'You are a professional image tagger.\nYour current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.\nYou will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more useful information, like character appearances and summaries of page groups.\nThe image alongside will be the panel itself. Here\'s an example of the metadata input:\n```\n{"title": "Jade: Keep going.", "content": "Oh yeah. How could you forget about his stash of DECREPIT MUMMIES.<br /><br />God you hate these things.", "page_id": "002753", "src": "/storyfiles/hs2/00853_1.gif", "page_group_summary": "Rose enters the Hubgrid.\nJade descends through Grandpa’s trophies.", "pov_characters": ["Jade"], "pov_commands": [["Jade: Proceed through house", "Jade: Be trolled by Karkat"]], "transcript_class": null, "transcri

# Textual datasets

We output textual datasets for Homestuck, MS Paint Adventures as a whole, and all of Andrew Hussie's works.

The format is .csv

In [ ]:
pass

# Visual datasets

We output visual datasets for Homestuck. (Note: https://huggingface.co/datasets/multimodalart/1920-raider-waite-tarot-public-domain, prioritize flux training?)
(https://replicate.com/ostris/flux-dev-lora-trainer/train for training styles, https://fal.ai/models/fal-ai/flux-lora-fast-training for training characters) 

In [157]:
"""
TODO: update when we actually have the complete dataset (i.e. tagged Homestuck act 5 onwards)
For now we'll go with four_acts_hs_df even though that's missing Hussnasty style and a bunch of characters

We can make LORAs for styles and characters since our content is almost perfectly tagged. However, LORAs are trained with natural language strings that use unusual strings. Instead of John Egbert, it probably wants something that never appeared in its datasets like hsjohnegbert or something. It also needs to be part of a sentence, like `a trtcrd of a woman wearing a crown of stars, sitting on a throne, holding a scepter, dressed in a robe with a pomegranate pattern, with a field and trees in the background, "the empress"`. 

These natural descriptions will require using GPT-4o once again, but this time with a LORA prompt, and then using the output of image/description pairs to train the LORA model.

Additionally, the special strings will need to be hardcoded, and have the model use them whenever the related tag appears and we're in the right dataset... if we're training a John Egbert model, we need to tell the model to use the special string in its description.

First of all, let's examine the tags the dataset actually uses. We can check the json file in the HSSE_TAGS constant for a hierarchy of tags. As a reminder, the file is organized like this:

```json
{
  "definitions": {
    "0": {
      "_id": 0,
      "name": "Character",
      "children": [
        1,
        32,
        56,
        60,
        104,
        132,
        148,
        155,
        173,
        184,
        253
      ]
    },
    "1": {
      "_id": 1,
      "name": "Human",
      "children": [
        2,
        15
      ]
    },
    "2": {
      "_id": 2,
      "name": "Kid",
      "children": [
        3,
        10
      ]
    },
    "3": {
      "_id": 3,
      "name": "Beta Kid",
      "children": [
        4,
        5,
        7,
        9
      ]
    },
    "4": {
      "_id": 4,
      "name": "John Egbert",
      "children": []
    },
...
```

Painfully, we can't actually determine which tags refer to a "final character". Rose has a child tag, "Rosesprite", so we can't rely on leaf elements.
We'll just have to play it by ear. We can start by getting all the tags that are children of "Character" and have been used more than 30 times.
"""

# Load the tags data
with open(HSSE_TAGS, 'r', encoding='utf-8') as f:
    tags_data = json.load(f)

# Function to get all nodes under a tag
def get_all_tag_members(tag_name: str) -> list:
    # Get the tag id from the tag name
    tag_id = [tag['name'] for tag in tags_data['definitions'].values()].index(tag_name)

    # Get the tag data
    tag = tags_data['definitions'][str(tag_id)]

    # Get the children of the tag
    children = tag['children']

    # If the tag has no children, return the tag name
    if not children:
        return [tag['name']]

    # Otherwise, get the members of the children
    members = [tag['name']]
    for child_id in children:
        members.extend(get_all_tag_members(tags_data['definitions'][str(child_id)]['name']))

    return members

character_tags = get_all_tag_members("Character")

# This is very annoying
organizational_tags = ["Character", "Human", "Kid", "Beta Kid", "Alpha Kid", "Guardian", "Alpha Guardian", "Real Person", "Troll", "Beta Troll", "Alpha Troll", "Cherub", "Felt", "Carapacian", "Exile", "Denizen", "Sprite", "Lusus", "Cat", "Horse", "Dog", "Frog", "Crow", "Bird", "Firefly", "Animal", "Insect", "Creature", "Angel", "Underling", "Drone", "Robot", "First Guardian", "Consort", "The Felt", "Exile", "Ancestor", "SBaHJ", "SbaHJ Drawing", "SBaHJifier", "Dersite"]

# Remove the organizational tags
character_tags = [tag for tag in character_tags if tag not in organizational_tags]
character_tags

['John Egbert',
 'Rose Lalonde',
 'Rosesprite',
 'Dave Strider',
 'Davesprite',
 'Jade Harley',
 'Jane Crocker',
 'Roxy Lalonde',
 'Dirk Strider',
 'Jake English',
 'Beta Guardian',
 'Dad Egbert',
 'Beta Bro',
 'Beta Mom',
 'Grandpa',
 'Nanna',
 'Nannasprite',
 'Dad Crocker',
 'Alpha Bro',
 'Alpha Mom',
 'Grandma',
 'Poppop',
 'Dad',
 'Dad Egbert',
 'Dad Crocker',
 'Bro',
 'Beta Bro',
 'Alpha Bro',
 'Mom',
 'Beta Mom',
 'Alpha Mom',
 'Liv Tyler',
 'Charles Dutton',
 'Charles Barkley',
 'Bill Cosby',
 'Bing Crosby',
 'Eddie Morton',
 'Nick Offerman',
 'Nicolas Cage',
 'Matthew McConaughey',
 'Snoop Dogg',
 'Mr. T',
 'Mr. T Puppet',
 'Foam Fetal Mr. T in a Jar',
 'Will Smith',
 'ICP',
 'Violent J',
 'Shaggy 2 Dope',
 'Guy Fieri',
 'Nick Cage',
 'Owen Wilson',
 'Ben Stiller',
 'Donald Glover',
 'Andrew Hussie',
 'Sweet Bro',
 'Hella Jeff',
 'Geromy',
 'SBaHJ Plush',
 'Karkat Vantas',
 'Aradia Megido',
 'Aradiasprite',
 'Aradiabot',
 'Tavros Nitram',
 'Tavrosprite',
 'GCatavrosprite',
 'So

In [155]:
# Works like a charm, but we need another function. Some character tags might have never been used, so let's extract all the rows that have a character tag and get all the unique characters 
# To do this efficiently, we'll first use pandas to extract all tags, combine all "tags" arrays into one that we can then parse

# Get all tags, sorted by number of appearances by exploding them all and then counting them
all_tags = four_acts_hs_df['tags'].explode().value_counts()

# Drop organizational/style ones
useful_tags = all_tags[~all_tags.index.str.contains("Part |Act | Mode")]

useful_tags


tags
John Egbert                516
John's House               400
Carapacian                 340
Green Slime Ghost Shirt    287
Computer                   282
                          ... 
Charles Barkley              1
Walking Cane                 1
Cancer Shirt                 1
Ms Paint                     1
Alt Calliope                 1
Name: count, Length: 557, dtype: int64

In [156]:
# Get all the character tags in the dataset if their count is over 30
character_tags_in_dataset = useful_tags[useful_tags.index.isin(character_tags) & (useful_tags > 30)].index.tolist()

# Remove repeats
character_tags_in_dataset = list(set(character_tags_in_dataset))

# Show some character tags
character_tags_in_dataset

['CD',
 'Doze',
 'Hearts Boxcars',
 'Harlequinsprite',
 'Diamonds Droog',
 'Rose Lalonde',
 'John Egbert',
 'HB',
 'Stitch',
 'Crowbar',
 'Serenity',
 'Jack Noir',
 'Dad Egbert',
 'Spades Slick',
 'Nanna',
 'Cans',
 'Imp',
 'Clubs Deuce',
 'Dave Strider',
 'Itchy',
 'Jaspers',
 'Eggs',
 'Midnight Crew',
 'Vodka Mutini',
 'DD',
 'PM',
 'Clover',
 'Dad',
 'Grandpa',
 'Sawbuck',
 'Dersite',
 'Jade Harley',
 'AR',
 'Ogre',
 'WV']

In [158]:
# From the given list, let's just keep the relevant ones I know we have enough data for:
# John Egbert, Rose Lalond, Dave Strider, Jade Harley, Dad, Spades Slick, Jack Noir, DD, Grandpa, PM, WV, Imp, Ogre 

CHARACTER_LORA_TAGS = ["John Egbert", "Rose Lalond", "Dave Strider", "Jade Harley", "Dad", "Spades Slick", "Jack Noir", "DD", "Grandpa", "PM", "WV", "Imp", "Ogre"]


In [159]:
# Now the function to create a LORA dataset ("image", "caption") for a specific character tag, then we'll just loop. 
# We'll use a "normalized" filename based on the tag name, like "johnegbert" for "John Egbert". 
# We'll assume that the given dataframe has a "caption" column that is the text description of the image, and a "src" column that contains a partial URL
# we can turn that into a full URL by prefixing "https://homestuck.com/images" to it as always
# Note that the final dataset will be csv with two columns, "image" and "caption

def create_lora_dataset_for_character_tag(df: pd.DataFrame, character_tag: str) -> pd.DataFrame:
    """
    Creates a LORA dataset for a specific character tag.

    Args:
        df (pd.DataFrame): The dataset to create the LORA dataset from.
        character_tag (str): The character tag to create the LORA dataset for.

    Returns:
        pd.DataFrame: The LORA dataset for the character tag.
    """
    # Get the normalized character tag
    normalized_character_tag = character_tag.lower().replace(" ", "")

    # Filter the dataset for the character tag
    character_df = df[df['tags'].apply(lambda tags: character_tag in tags)]

    # Create the LORA dataset
    lora_dataset = character_df[['src', 'content']].copy()
    lora_dataset['src'] = lora_dataset['src'].apply(lambda src: f"https://homestuck.com/images{src}")
    # TODO: replace "content" with an actual caption column later
    lora_dataset.rename(columns={'src': 'image', 'content': 'caption'}, inplace=True)

    # Save it as a csv in OUTPUT_FOLDER
    lora_dataset_output_file = os.path.join(OUTPUT_FOLDER, f"lora_{normalized_character_tag}.csv")
    lora_dataset.to_csv(lora_dataset_output_file, index=False)
    print(f"LORA dataset for {character_tag} saved to {lora_dataset_output_file}")
    
    return lora_dataset

# Test it with John Egbert
john_egbert_lora_dataset = create_lora_dataset_for_character_tag(four_acts_hs_df, "John Egbert")
john_egbert_lora_dataset.sample(5)

LORA dataset for John Egbert saved to ../output\lora_johnegbert.csv


,image,caption
4402,https://homestuck.com/images/storyfiles/hs2/01...,<NA>
2830,https://homestuck.com/images/storyfiles/hs2/00...,You plug in your LAPTOP and connect to the int...
2473,https://homestuck.com/images/storyfiles/hs2/00...,<NA>
3591,https://homestuck.com/images/storyfiles/hs2/01...,John is of course sound asleep. It looks like ...
2686,https://homestuck.com/images/storyfiles/hs2/00...,"You're not sure you even saw a woman, let alon..."


In [ ]:
# TODO: Now we can loop through the character tags and create a LORA dataset for each one
pass

In [160]:
# Now let's make style LORAs, though we already know which tags we want, sprite mode, hero mode, lined mode, and scribble mode (TODO: homosuck and hussnasty)
# We'll just filter the dataset for each style and create a LORA dataset for it

# First, let's get the tags for the styles
style_tags = ["Sprite Mode", "Hero Mode", "Lined Mode", "Scribble Mode"]

def create_lora_dataset_for_style_tag(df: pd.DataFrame, style_tag: str) -> pd.DataFrame:
    """
    Creates a LORA dataset for a specific style tag.

    Args:
        df (pd.DataFrame): The dataset to create the LORA dataset from.
        style_tag (str): The style tag to create the LORA dataset for.

    Returns:
        pd.DataFrame: The LORA dataset for the style tag.
    """
    # Get the normalized style tag
    normalized_style_tag = style_tag.lower().replace(" ", "")

    # Filter the dataset for the style tag
    style_df = df[df['tags'].apply(lambda tags: style_tag in tags)]

    # Create the LORA dataset
    # TODO: once again, we should have an actual caption here
    lora_dataset = style_df[['src', 'content']].copy()
    lora_dataset['src'] = lora_dataset['src'].apply(lambda src: f"https://homestuck.com/images{src}")
    
    return lora_dataset

# Test it with Sprite Mode
sprite_mode_lora_dataset = create_lora_dataset_for_style_tag(four_acts_hs_df, "Sprite Mode")
sprite_mode_lora_dataset.sample(5)

,src,content
2419,https://homestuck.com/images/storyfiles/hs2/00...,You can now execute that brilliant idea you ha...
3703,https://homestuck.com/images/storyfiles/hs2/01...,<NA>
4508,https://homestuck.com/images/storyfiles/hs2/01...,<NA>
4115,https://homestuck.com/images/storyfiles/hs2/01...,<NA>
2435,https://homestuck.com/images/storyfiles/hs2/00...,"You play the prankster's favorite card game, e..."
